# Проект: Предиктивная модель оценки стоимости недвижимости

## 1. Описание кейса и бизнес-постановка
К нам обратился представитель крупного агентства недвижимости.

**Проблема:** Риелторы тратят слишком много времени на ручную сортировку объявлений и поиск выгодных предложений. Скорость реакции и качество ручной оценки уступают конкурентам, что негативно сказывается на финансовых показателях агентства.

**Цель проекта:**

Разработать сервис (модель машинного обучения), который позволит **автоматически предсказывать рыночную стоимость объектов недвижимости** на основе их характеристик и истории предложений. Это поможет риелторам быстрее находить недооцененные варианты и точнее формировать предложения для клиентов.

## 2. Поставленные задачи
Для достижения цели необходимо решить следующие технические и аналитические задачи:

1.  **Разведочный анализ данных (EDA) и Очистка:**
    *   Провести парсинг сложных вложенных структур (JSON) в признаках `homeFacts` и `schools`.
    *   Выявить и обработать аномальные выбросы (ошибки ввода площади, нереалистичные цены), которые могут исказить обучение.
    *   Проанализировать распределения и корреляции признаков.

2.  **Feature Engineering (Проектирование признаков):**
    *   Восстановить и использовать географические данные (`latitude`, `longitude`, `zipcode`).
    *   Применить продвинутые методы кодирования категорий (Target Encoding) для признаков с высокой кардинальностью.
    *   Сформировать новые информативные признаки (возраст дома, расстояние до школ, средний рейтинг района).

3.  **Построение ML-пайплайна:**
    *   Разработать воспроизводимый `sklearn.pipeline`, включающий обработку пропусков, масштабирование и кодирование.
    *   **Критически важно:** Исключить утечку данных (Data Leakage) — проводить все трансформации строго после разделения на обучающую и тестовую выборки.

4.  **Моделирование и Оптимизация:**
    *   Построить Baseline-модель для оценки базового качества.
    *   Провести автоматический подбор гиперпараметров с помощью **Optuna**.
    *   Реализовать ансамблевые методы (**Stacking**) с использованием **GPU-ускорения** для достижения максимальной точности.

5.  **Внедрение:**
    *   Подготовить и сериализовать (сохранить) лучшую модель для дальнейшей интеграции в веб-сервис.

## 3. Описание данных
В работе используется [датасет](https://drive.google.com/file/d/11-ZNNIdcQ7TbT8Y0nsQ3Q0eiYQP__NIW/view?usp=share_link), содержащий ~377 000 записей о продаже недвижимости.
Ключевые признаки:
*   **Целевая переменная:** `target` (стоимость объекта недвижимости).
*   **Характеристики:** площадь (`sqft`, `lotsize`), количество комнат (`beds`, `baths`), этажность, тип недвижимости.
*   **Локация:** адрес, город, штат, почтовый индекс, координаты.
*   **Инфраструктура:** данные о школах, рейтинги, расстояния.

In [ ]:
# 1. НАСТРОЙКА ОКРУЖЕНИЯ И ИМПОРТ БИБЛИОТЕК
import os
import sys
import warnings
import datetime
import time
import math
import pickle
import joblib
import ast
import logging


# --- Intel Acceleration (CPU) ---
try:
    from sklearnex import patch_sklearn
    patch_sklearn()
    print('Intel Extension (CPU) активирован')
except (ImportError, Exception):
    print('Intel Extension не найден, работаем на стандартном CPU')

# ==============================================================================================
# 2. НАСТРОЙКА GPU (NVIDIA)
# ==============================================================================================
# Флаг пользователя: Хотим ли мы использовать GPU, если она есть?
ENABLE_GPU_COMPUTING = True 

try:
    import torch
    # Проверяем, видит ли PyTorch видеокарту
    REAL_GPU_AVAILABLE = torch.cuda.is_available()
    GPU_NAME = torch.cuda.get_device_name(0) if REAL_GPU_AVAILABLE else "Нет устройства"
except ImportError:
    REAL_GPU_AVAILABLE = False
    GPU_NAME = "PyTorch не установлен"

# Итоговое решение: Используем GPU только если пользователь хочет И она физически есть
USE_GPU = ENABLE_GPU_COMPUTING and REAL_GPU_AVAILABLE

if USE_GPU:
    print(f'GPU активирована: {GPU_NAME}')
else:
    if ENABLE_GPU_COMPUTING and not REAL_GPU_AVAILABLE:
        print(f'Вы запросили GPU, но она не найдена ({GPU_NAME}). Переключаемся на CPU.')
    else:
        print('Работаем на CPU (GPU отключена в настройках)')


# 3. ИМПОРТ DATA SCIENCE БИБЛИОТЕК

# Работа с данными
import pandas as pd
import numpy as np
#from tqdm import tqdm

# Визуализация
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# Статистика (Тесты гипотез)
from scipy import stats
from scipy.stats import f_oneway, kruskal # <--- НУЖНО ДЛЯ ANOVA
import scikit_posthocs as sp # <--- НУЖНО ДЛЯ ТЕСТА ДАННА

# Интерактивный анализ (D-Tale)
# %pip install dtale 
import dtale
import dtale.app as dtale_app

# Machine Learning (Sklearn Modules - для поддержки старого кода)
from sklearn import (
    model_selection,
    preprocessing,
    linear_model,
    metrics,
    tree,
    ensemble,
    feature_selection
)

# Machine Learning (Direct Imports - для пайплайнов)
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    StackingRegressor, 
    GradientBoostingRegressor, 
    RandomForestRegressor, 
    HistGradientBoostingRegressor,
    AdaBoostRegressor
)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression

# Boosting (работают и на CPU, и на GPU)
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import xgboost as xgb

# Инструменты ML
import optuna
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence
from category_encoders import TargetEncoder 
import pgeocode # Для геоданных

# 4. НАСТРОЙКИ ЛОГИРОВАНИЯ И ОТОБРАЖЕНИЯ

# Настройки Pandas и Plotly
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8') 
sns.set_style("whitegrid")
pio.renderers.default = 'notebook' # Для отображения в Jupyter / GitHub

# Подавление предупреждений
warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Логирование в файл
if not os.path.exists('./logs'):
    os.makedirs('logs')

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"logs/experiments_{timestamp}.log"

logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S',
    force=True 
)

# Фильтр шума от библиотек
logging.getLogger('sklearnex').setLevel(logging.WARNING)
logging.getLogger('optuna').setLevel(logging.WARNING)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('ipykernel').setLevel(logging.ERROR)
logging.getLogger('comm').setLevel(logging.ERROR)

print(f"Логирование настроено в файл: {log_filename}")
logging.info("Старт сессии")

# Глобальные константы
RANDOM_SEED = 42
EXPERIMENTS_DATA = [] # Список для сбора результатов

## Загрузка данных, Разведочный анализ данных (EDA) и Очистка

In [ ]:
# загружаем данные
df_estate = pd.read_csv(r'./data/data.csv')
df_estate.head()

В датасете имеем 377185 разных объектов недвижимости описанных восемнадцатью признаками:

1. status - статус продажи
2. private pool - частный бассейн
3. propertyType - тип недвижимости
4. street - улица
5. baths - число ванных комнат 
6. homeFacts - сведения о доме 
7. fireplace - тип отопления
8. city - город
9. schools - школы
10. sqft - квадратный фут
11. zipcode - почтовый индекс
12. beds - число комнат
13. state - штат
14. stories - этажность
15. mls-id - это код в их централизованной системе учёта предложений объектов недвижимости
16. PrivatePool - частный бассейн
17. MlsId - это код в их централизованной системе учёта предложений объектов недвижимости
18. target - целевой признак, цена недвижимости


## 1. Первичный анализ и очистка данных

Проведем осмотр данных, проверим типы, наличие пропусков и дубликатов.

In [ ]:
df_estate.info()

Все признаки имеют тип object, в том числе целевой.  Кроме того, имеются пропуски во всех признаках. 

Сформилируем дальнейшую схему работы/обработки данных на следующем этапе:

1. Привести целевой признак(target) к числовому типу
2. Обработать пропуски в целевом признаке(target)
3. Проанализировать и удалить неинформативные признаки
4. Отдельно обработать информацию в признаках homeFacts и school, содержащие пул данных
5. Обработать пропуски в остальных признаках
6. Спроектировать новые признаки
7. Визуализировать данные, найти зависимости, гипотезы

In [ ]:
print(f"Размер датасета: {df_estate.shape}")
duplicates = df_estate.duplicated().sum()
print(f"Количество дубликатов: {duplicates} ({duplicates/len(df_estate):.2%})")

### Удаление дубликатов
Удалим дубликаты, т.к. они могут исказить результаты анализа и обучения модели.

In [ ]:
df_estate = df_estate.drop_duplicates()
print(f"Размер после удаления дубликатов: {df_estate.shape}")

### Обработка целевой переменной (target)
Целевая переменная сейчас в строковом формате (содержит '$' и ','). Преобразуем ее в число.

In [ ]:
def clean_currency(x):
    if isinstance(x, str):
        return x.replace('$', '').replace(',', '').replace('+', '')
    return x

df_estate['target_clean'] = df_estate['target'].apply(clean_currency)
df_estate['target_clean'] = pd.to_numeric(df_estate['target_clean'], errors='coerce')
# удаляем признак target
df_estate = df_estate.drop(['target'], axis=1)

print(f'Число пропусков в target_clean: {df_estate["target_clean"].isnull().sum()}.')

Имеем 2878 пропусков в целевой переменной, что составляет менее 1% от общего объёма датасета. 

Удалим строки с пропущенными значениями целевой переменной. Мы учим модель находить зависимость между характеристиками дома и реальной ценой. Если мы заполняем пропуски медианой, мы подсовываем модели "синтетику". Модель будет учиться предсказывать не рыночную цену, а рассчитаюнную медиану. Если мы заполним пропуски в target и эти строки попадут в тестовую выборку (test), то при проверке качества мы будем проверять, насколько хорошо модель угадала нашу же медиану, а не реальность.


In [ ]:
# Удаляем строки, где в целевой переменной (target_clean) есть пропуски (NaN)
df_estate = df_estate.dropna(subset=['target_clean'])

# (Опционально) Сбрасываем индекс, чтобы он шел по порядку после удаления строк
df_estate = df_estate.reset_index(drop=True)
print(f'Количество пропусков целевой переменной = {df_estate["target_clean"].isnull().sum()}.')
print(f'Текущий размер датасета: {df_estate.shape}')

Описательные статистики по целевой переменной(target_clean):

In [ ]:
print("Основные статистики Price:")
print(df_estate['target_clean'].describe())

Медианная стоимость недвижимости $320 000 
Средняя стоимость недвижимости $645 409 , что превышаем медианную стоимость в 2 раза.
Кроме того, предположительно имеют место выбросы (минимальная стоимость $1, максимальная стоимость $195 000 000).
В таком случае именно медианное значение цены будет лучше отражать меру центральной тенденции так как оно устойчиво к выбросам. 

Изучим распределение целевой переменной:

In [ ]:

fig = px.histogram(
    df_estate, 
    x='target_clean', 
    nbins=60, 
    marginal='rug', 
    title='Распределение стоимости недвижимости',
    labels={'target_clean': 'Цена ($)'},
    template='plotly_white'
)

fig.update_layout(bargap=0.1)
fig.show()

Исходная гистограмма цен крайне неинформативна: из-за экстремальных выбросов (до $2 \times 10^8$) основное распределение "схлопнулось" в узкий пик около нуля. Чтобы избавиться от влияния этого длинного хвоста и сделать структуру данных видимой, мы применили логарифмическую трансформацию к целевой переменной



In [ ]:
df_estate['target_clean_log'] = np.log(df_estate['target_clean'])


fig = px.histogram(
    df_estate, 
    x='target_clean_log', 
    nbins=60, 
    # kde=True,  
    marginal='box', 
    title='Распределение стоимости недвижимости (логарифмическая трансформация)',
    labels={'target_clean_log': 'Log(Цена)'},
    color_discrete_sequence=['#636EFA'],
    template='plotly_white'
)
fig.update_layout(bargap=0.1)
fig.show()

Избавимся от выбросов по целевому признаку, для этого воспользуемся методом Тьюки, применив его к логарифмированному признаку:

In [ ]:
def find_outliers_iqr(df, feature, left=1.5, right=1.5, log_scale=False):
    """
    Находит выбросы в данных, используя метод межквартильного размаха. 
    Классический метод модифицирован путем добавления:
    * возможности логарифмирования распредления
    * ручного управления количеством межквартильных размахов в обе стороны распределения
    Args:
        df (pandas.DataFrame): набор данных
        feature (str): имя признака, на основе которого происходит поиск выбросов
        left (float, optional): количество межквартильных размахов в левую сторону распределения. По умолчанию 1.5.
        right (float, optional): количество межквартильных размахов в правую сторону распределения. По умолчанию 1.5.
        log_scale (bool, optional): режим логарифмирования. По умолчанию False - логарифмирование не применяется.

    Returns:
        pandas.DataFrame: наблюдения, попавшие в разряд выбросов
        pandas.DataFrame: очищенные данные, из которых исключены выбросы
    """
    if log_scale:
        x = np.log(df[feature]+1)
    else:
        x= df[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = df[(x<lower_bound) | (x > upper_bound)]
    cleaned = df[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

print(f'Тип ассиметрии до очистки: {df_estate["target_clean_log"].skew()}')
outliers, df_estate = find_outliers_iqr(df_estate, 'target_clean_log')
print(f'Тип ассиметрии после очистки: {df_estate["target_clean_log"].skew()}')

In [ ]:
fig = px.histogram(
    df_estate, 
    x='target_clean_log', 
    nbins=60, 
    # kde=True,  
    marginal='box', 
    title='Распределение стоимости недвижимости (логарифмическая трансформация)',
    labels={'target_clean_log': 'Log(Цена)'},
    color_discrete_sequence=['#636EFA'],
    template='plotly_white'
)
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
fig = px.histogram(
    df_estate, 
    x='target_clean', 
    nbins=60, 
    # kde=True,  <-- Удаляем эту строку
    marginal='box', 
    title='Распределение стоимости недвижимости (логарифмическая трансформация)',
    labels={'target_clean': 'Цена ($)'},
    color_discrete_sequence=['#636EFA'],
    template='plotly_white'
)
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
print(f'Число наблюдений {df_estate.shape[0]}.')
df_estate['target_clean'].describe()

In [ ]:
df_estate.info()

После очистки датасета количество наблюдений уменьшилось до 348064, аномальные значения в виде $1 или $195 млн. были удалены. 

## Анализ неинформативных признаков

Изучим потенциальные неинформативные признаки 
- наличие частного бассейна - private pool 
- наличие частного бассейна - PrivatePool, 
- наличие каминов - fireplace 

- в них порядка 65-85% пропусков и они кандидаты на удаление, 

признаки MlsId и mls-id, которые являются уникальными номерами в базе данных в системе множественного листинга и никакой информативности не несут.


In [ ]:
display(df_estate['private pool'].describe())
display(df_estate['private pool'].value_counts())

In [ ]:
display(df_estate['PrivatePool'].describe())
display(df_estate['PrivatePool'].value_counts())

In [ ]:
display(df_estate['fireplace'].describe())
display(df_estate['fireplace'].value_counts())

In [ ]:
def date_clear(df, no_inf_1, no_inf_2):
    """ Удаляет столбцы, в которых доля пропущенных значений > 60%,
    а также неинформативные признаки

    Args:
        df (DateFrame): исходный датафрейм
        no_inf_1 (Series): первый неинформативный признак
        no_inf_2 (Series): второй неинформативный признак

    Returns:
        DateFrame: очищенный датафрейм
    """
    df = df.drop([no_inf_1, no_inf_2], axis=1)
    #задаем минимальный порог: вычисляем 60% от числа строк
    thresh = df.shape[0]*0.6
    #удаляем столбцы, в которых более 30% (100-70) пропусков
    df = df.dropna(thresh=thresh, axis=1)
    return df

df_estate = date_clear(df_estate, 'mls-id', 'MlsId')
df_estate.info()

### **status(статус продажи), propertyType(тип недвижимости), street(улица)**

В признаках статус продажи(status), тип недвижимости(propertyType) и улица(street) заполним пропуски самым распространённым значением - for sale, single-family home и Address Not Disclosed соответственно:

In [ ]:
df_estate['status'].describe()

In [ ]:
df_estate['status'] = df_estate['status'].fillna(df_estate['status'].describe().top)

In [ ]:
df_estate['propertyType'].describe()

In [ ]:
df_estate['propertyType'] = df_estate['propertyType'].fillna(df_estate['propertyType'].describe().top)

In [ ]:
df_estate['street'].describe()

In [ ]:
df_estate['street'] = df_estate['street'].fillna(df_estate['street'].describe().top)

Для удобства работы переведём данные в признаках статус продажи(status), тип недвижимости(propertyType), улицу(street) в нижний регистр:

In [ ]:
df_estate['status'] = df_estate['status'].apply(lambda x: x.lower())
df_estate['propertyType'] = df_estate['propertyType'].apply(lambda x: x.lower())
df_estate['street'] = df_estate['street'].apply(lambda x: x.lower())

Если вглянуть на статусы, то основная часть покрывается статусами:
- for sale(продаётся)
- active(активное)
- foreclosure(обращение взыскания)
- new construction()
- pending(в ожидании)

Оставим данные статусы, остальные переименуем в категорию other(другие).

In [ ]:
df_estate['status'].value_counts()[:5]

In [ ]:
# лист с категориями
status_list = list(df_estate['status'].value_counts()[:5].index)
df_estate['status'] = df_estate['status'].apply(lambda x: x if x in status_list else 'other')

Аналогично посмотрим на тип недвижимости(propertyType):

- single family(дом на одну семью)
- condo(кондоминиум)
- land(земля)
- townhouse(таунхаус)
- multi family(для многодетной семьи/большой семьи)
- high rise(в выстоном здании)
- ranch(ранчо)
- coop(кооперативные квартиры/дома)
- single detached(отдельно стоящий дом)
- traditional(стандартное)

Всё, что не попадает в вышеуказанные категории будут заменены на other.

In [ ]:
df_estate['propertyType'].value_counts()[:20]

In [ ]:
# проводим удаление(замену) ненужных символов 
df_estate['propertyType'] = df_estate['propertyType'].apply(lambda x: ' '.join(x.replace('-', ' ').replace(',', ' ').replace('/', ' ').split(' ')[0:2]))
df_estate['propertyType'] = df_estate['propertyType'].replace('condo townhome', 'condo').replace('lot land', 'land').replace('detached ', 'single detached')
# лист с категориями
propertyType_list = list(df_estate['propertyType'].value_counts()[:11].index)
# удаляем одну неинформационную категорию 
propertyType_list.remove('1 story')
df_estate['propertyType'] = df_estate['propertyType'].apply(lambda x: x if x in propertyType_list else 'other')

Преобразуем признак улица(street), избавившись от номеров домов и оставив только наименование улиц - подразумевая, что стоимость недвижимость привязана к локации, а наименование улицы один из способов привязаться к этому местоположению(району города). Далее те улицы, которые будут в единственном экземпляре объединим в категорию other.

In [ ]:
df_estate['street'] = df_estate['street'].replace('undisclosed address', 'address not disclosed').replace('(undisclosed address)', 'address not disclosed')\
    .replace('address not available', 'address not disclosed').replace('unknown address', 'address not disclosed')
    
df_estate['street_new'] = df_estate['street'].apply(lambda x: ' '.join(x.split(' ')[1:4])).replace('', 'address not disclosed').replace('address not disclosed', 'not disclosed')

df_estate = df_estate.drop(['street'], axis=1)

In [ ]:
street_list = list(df_estate['street_new'].value_counts()[df_estate['street_new'].value_counts()==1].index)
date_street_one = df_estate[df_estate['street_new'].isin(street_list)]
date_street_one['street_new'] = date_street_one['street_new'].apply(lambda x: 'other')
df_estate = pd.concat(
    [df_estate[~df_estate['street_new'].isin(street_list)], date_street_one],
    ignore_index=True,
    axis=0
)

### **city(город)**

Следующий признак - город(city), в котором имеются 20 пропущенных значений:

In [ ]:
df_estate['city'].isnull().sum()

#### СТРАТЕГИЯ ЗАПОЛНЕНИЯ ПРОПУСКОВ ГЕОДАННЫХ:
1. Если в исходнике нет города/штата/координат, берем их из pgeocode для надежности.
2. Если и там нет - это мусорные данные

ВАЖНО: Мы НЕ заполняем координаты средним значением. Локация - это ключевой драйвер цены. "Средняя координата" (центр США) для дома во Флориде создаст чудовищный шум в модели. Лучше удалить эти строки (их обычно < 1%), чем кормить модель ложью.

In [ ]:
import pgeocode

# 0. ЗАЩИТА ОТ ПОВТОРНОГО ЗАПУСКА (ОЧИСТКА)
# Если мы перезапускаем ячейку, удаляем следы предыдущих попыток, чтобы merge не создавал дубликатов (_x, _y)
temp_cols = ['zipcode_clean', 'city_geo', 'county_geo', 'state_geo', 'lat_geo', 'lon_geo']
df_estate = df_estate.drop([c for c in temp_cols if c in df_estate.columns], axis=1)

# 1. ПОДГОТОВКА И ОЧИСТКА ZIP-КОДОВ
def clean_zip_code(x):
    x = str(x).strip()
    if x == '--' or x == '0' or pd.isna(x) or x == 'nan':
        return None
    return x.split('-')[0]

df_estate['zipcode_clean'] = df_estate['zipcode'].apply(clean_zip_code)

# 2. ГЕНЕРАЦИЯ ГЕОДАННЫХ (pgeocode)
print("Запрос актуальных геоданных через pgeocode...")
nomi = pgeocode.Nominatim('us')

unique_zips = df_estate['zipcode_clean'].dropna().unique().astype(str).tolist()

# Запрос данных
geo_info = nomi.query_postal_code(unique_zips)

# Отбираем нужные колонки
geo_ref = geo_info[['postal_code', 'place_name', 'county_name', 'state_code', 'latitude', 'longitude']].copy()
geo_ref.columns = ['zipcode_clean', 'city_geo', 'county_geo', 'state_geo', 'lat_geo', 'lon_geo']

# 3. ОБЪЕДИНЕНИЕ И УТОЧНЕНИЕ ДАННЫХ
df_estate = df_estate.merge(geo_ref, on='zipcode_clean', how='left')

# СТРАТЕГИЯ ЗАПОЛНЕНИЯ:
mappings = [
    ('latitude', 'lat_geo'), 
    ('longitude', 'lon_geo'), 
    ('city', 'city_geo'), 
    ('state', 'state_geo'), 
    ('county', 'county_geo')
]

for col, geo_col in mappings:
    if col in df_estate.columns:
        # Если колонка уже была (например, city), заполняем только пропуски
        df_estate[col] = df_estate[col].fillna(df_estate[geo_col])
    else:
        # Если колонки не было (latitude), создаем её из гео-данных
        df_estate[col] = df_estate[geo_col]

# Удаляем вспомогательные колонки
df_estate = df_estate.drop(temp_cols, axis=1, errors='ignore')

# 4. ОБРАБОТКА ОСТАВШИХСЯ ПРОПУСКОВ
missing_coords = df_estate['latitude'].isnull().sum()
print(f"Строк без координат после обогащения данными: {missing_coords}")

if missing_coords > 0:
    print(f"Удаляем {missing_coords} строк с неизвестной локацией...")
    df_estate = df_estate.dropna(subset=['latitude', 'longitude'])

# Сбрасываем индекс для порядка
df_estate = df_estate.reset_index(drop=True)

print(f"Итоговый размер датасета: {df_estate.shape}")

In [ ]:
df_estate.info()

### **baths(число ванных комнат)**

Займёмся признаком число ваных(baths) - данный признак содержит разные строковые значения и символы, которые требуют обработки, также имеется в числовых значениях(представленых в виде строкового) имеется символ ',' вместо точки, его необходимо заменить на точку для корректного преобразования в числовой формат(float). Пропущенные значения заменим на медианное.

In [ ]:
def handler_baths(df):
    """ Обработчик признака baths

    Args:
        df (DateFrame): входящий датафрейм

    Returns:
        DateFrame: обработанный датафрейм
    """
    # проверяем на пропущенное значение
    if df  is not np.nan:
        # производим замену симвволов, производим сплит, преобразуем в лист
        df = df.replace(',', '.').split(' ')
        # проходим по каждому элементу листа
        for i in df:
            try:
                i = float(i)
                return i
            # обрабатываем исключение - если элемент невозможно преобразовать в float, то пропускаем
            except ValueError:
                continue  
    else:
        return df
    
df_estate['baths'] = df_estate['baths'].apply(handler_baths)
df_estate['baths'] = df_estate['baths'].fillna(df_estate['baths'].median())

После преобразования имеем:

In [ ]:
df_estate['baths'].describe()

Сразу смущает максимальные значения в 750, скорее всего это выбросы, которые будут сбивать модель, избавимся от них ограничив размер 20. Нулевые значения оставляем, так как в датафрейме имеются тип недвижимости земля(land), что вполне объясняет нулевые значения.

In [ ]:
df_estate = df_estate[df_estate['baths'] < 21].reset_index().drop(['index'], axis=1)

In [ ]:
df_estate.shape

### **sqft(квадратный фунт)**

Возьмёмся за признак **квадратный фут(sqft)** - напишем функцию-обработчик, обратим внимание, что в значениях имеются записи в формате: '--', '2500 sqft', '-- sqft', 'Total interior livable area: 2,533 sqft', а также пропущенные значения, которые заменим на медианное.

In [ ]:
df_estate['sqft'].value_counts()[:20]

In [ ]:
def handler_sqft(val):
    """ Обрабатывает признак sqft """
    
    # 1. Проверка на NaN (используем pd.isna для надежности) или явные пропуски
    if pd.isna(val) or val == '--':
        return np.nan
    
    # Превращаем в строку и убираем запятые сразу для всех проверок
    val = str(val).replace(',', '').strip()
    
    # 2. Проверка на мусорные строки, начинающиеся с тире
    if val.startswith('--'):
        return np.nan
    
    # 3. ОБРАБОТКА ДИАПАЗОНА (исправление для '610-840')
    if '-' in val:
        try:
            # Разбиваем '610-840' на ['610', '840'], переводим в числа и считаем среднее
            parts = [float(x) for x in val.split('-')]
            return sum(parts) / len(parts)
        except ValueError:
            pass # Если не получилось, идем дальше
            
    # 4. Обработка формата 'Total interior livable area: ...'
    if 'total' in val.lower():
        try:
            # Берем предпоследний элемент
            return float(val.split()[-2])
        except (ValueError, IndexError):
            return np.nan

    # 5. Стандартный случай ('1200 sqft' -> берем первое слово)
    try:
        return float(val.split(' ')[0])
    except ValueError:
        # Если ничего не помогло (например, совсем битая строка), возвращаем NaN
        return np.nan

# Применяем функцию
df_estate['sqft_new'] = df_estate['sqft'].apply(handler_sqft)

# Заполняем пропуски медианой
df_estate['sqft_new'] = df_estate['sqft_new'].fillna(df_estate['sqft_new'].median())

# Удаляем старый столбец
df_estate = df_estate.drop(['sqft'], axis=1)

# Проверяем результат
df_estate['sqft_new'].describe()

### **stories(число этажей)**

Следующий признак - **число этажей(stories)** - в нём данные представлены в строковом формате, некоторые признаки прописаны словами, так, например, первый этаж может быть прописан 'One', 'One,' '1 Story', '1.0', есть пропущенные значения, спец. символы, например, '+', записи не содержащие никакие отсылки к этажности(неинформативные) - их заменим на медианное значение.

In [ ]:
df_estate['stories'].value_counts()[:20]

In [ ]:
def handler_stories(df):
    """ Обработчик признака stories

    Args:
        df (Series): необработанный признак

    Returns:
        float/np.nan: обработанный признак
    """
    # словарь для замены номеров этажей
    stories_dict = {'One':1, 'Two':2, 'Three':3, 'Tri-Level':3, 'One,':1, '1-2':1.5, 'Bi-Level':2, '3-4':3.5, 'Two,':2, 'Tri':3}
    
    if df is not np.nan:
        df = df.replace('+', '').split(' ')[0]
        if df in stories_dict.keys():
            df = stories_dict[df]
            try:
                df = float(df)
                return df
            except ValueError:
                df = np.nan
                return df
        else:
            try:
                df = float(df)
                return df
            except ValueError:
                df = np.nan
                return df
    else:
        return df
    
df_estate['stories_new'] = df_estate['stories'].apply(handler_stories)
df_estate = df_estate.drop(['stories'], axis=1)

Обратим внимание на два объекта с этажностью значением более 100, явно это выбросы от которых необходимо избавиться:

In [ ]:
df_estate[df_estate['stories_new'] > 100]

In [ ]:
df_estate['stories_new'] = df_estate['stories_new'].fillna(df_estate['stories_new'].median())
df_estate = df_estate[df_estate['stories_new'] < 100].reset_index().drop(['index'], axis=1)
df_estate['stories_new'].describe()

### **beds(число комнат)**

In [ ]:
def handler_beds(df):
    """ Обработчик признака beds

    Args:
        df (Series): необработанный признак

    Returns:
        float/np.nan: обработанный признак
        """
    beds_list = list(range(0,16))
    if df is not np.nan:
        try:
            df = float(df.replace(',', '.').split(' ')[0])
            if df in beds_list:
                return df
            else:
                return np.nan  
        except ValueError:
            return np.nan    
    else:
        return df
    
df_estate['beds_new'] = df_estate['beds'].apply(handler_beds)      
df_estate = df_estate.drop(['beds'], axis=1)
df_estate['beds_new'] = df_estate['beds_new'].fillna(df_estate['beds_new'].median())    

### **Сведения о доме(homeFacts)**

Займёмся признаком сведения о доме(homeFacts) - в котором намешано много информации и она представлена в строковом формате, который в свою очередь похож на json-формат(данные хранятся в виде словарей и списков). Внимательно взглянем на информацию, содержащуюся в данном признаке:

- atAGlanceFacts(факты) - основной ключ, значениями которого является список словарей

В каждом словаре находятся два ключа -**factLabel**(наименование параметра/факта) и **factValue**(значение параметра). 

Имеем следующие виды фактов(factValue):

- Year built(год постройки дома)
- Remodeled year(год реконструкции)
- Heating(отопление)
- Cooling(охлаждение)
- Parking(паркинг)
- lotsize(дом с участком)
- Price/sqft(цена за квадратный фунт)

Задача - разнести все параметры по отдельным признакам, далее преобразовать их, избавиться от выбросов.

In [ ]:
# преобразование строки в словарь
df_estate['homeFacts_new'] = df_estate['homeFacts'].apply(lambda x: ast.literal_eval(x))
# создание новых признаков
df_estate['year_built'] = df_estate['homeFacts_new'].apply(lambda x: x['atAGlanceFacts'][0]['factValue'])
df_estate['remodeled_year'] = df_estate['homeFacts_new'].apply(lambda x: x['atAGlanceFacts'][1]['factValue'])
df_estate['heating'] = df_estate['homeFacts_new'].apply(lambda x: x['atAGlanceFacts'][2]['factValue'])
df_estate['cooling'] = df_estate['homeFacts_new'].apply(lambda x: x['atAGlanceFacts'][3]['factValue'])
df_estate['parking'] = df_estate['homeFacts_new'].apply(lambda x: x['atAGlanceFacts'][4]['factValue'])
df_estate['lotsize'] = df_estate['homeFacts_new'].apply(lambda x: x['atAGlanceFacts'][5]['factValue'])
df_estate['price_sqft'] = df_estate['homeFacts_new'].apply(lambda x: x['atAGlanceFacts'][6]['factValue'])
# удаление первначального признака
df_estate = df_estate.drop(['homeFacts'], axis=1)

#### **year_built(год постройки дома)**

Взглянем на year_built:

In [ ]:
df_estate['year_built'].isnull().sum()

In [ ]:
df_estate['year_built'].value_counts()[:20]

имеем 3092(+44584) незаполненных значений. Преобразуем признак в числовой формат и заполним пропуски медианным значением. Также заменим на медианные значения дома года постройки > 2025 года и раньше 1700 года.

In [ ]:
df_estate['year_built'] = df_estate['year_built'].apply(lambda x: np.nan if x == '' else np.nan if x == 'No Data' else np.nan if x is None else x)
df_estate['year_built'] = df_estate['year_built'].apply(lambda x: int(x) if x is not np.nan else x)
df_estate['year_built'] = df_estate['year_built'].fillna(df_estate['year_built'].median())
df_estate['year_built'] = df_estate['year_built'].apply(lambda x: df_estate['year_built'].median() if x > 2025 else df_estate['year_built'].median() if x < 1700 else x)

In [ ]:
df_estate['year_built'].describe()

Вместо года постройки создадим новый признак - возраст дома(age_home) - разницой текущего года и года постройки дома.

In [ ]:
# текущий год
year_current = datetime.datetime.now().year
df_estate['age'] = year_current - df_estate['year_built']

### **remodeled_year(год реконструции)**

In [ ]:
df_estate['remodeled_year'].isnull().sum()

In [ ]:
df_estate['remodeled_year'] = df_estate['remodeled_year'].apply(lambda x: np.nan if (x == '') | (x == None) else x)
df_estate['remodeled_year'] = df_estate['remodeled_year'].apply(lambda x: int(x) if x is not np.nan else x)

In [ ]:
df_estate['remodeled_year'].describe()

В признаке 24898 пропуска, заменим их на медианное значение - 1986:

In [ ]:
df_estate['remodeled_year'] = df_estate['remodeled_year'].fillna(df_estate['remodeled_year'].median())

Сразу стоит проверить - дата реконструции не может быть раньше(либо равно) даты постройки, поэтому все значения в признаке remodeled_year, которые не будут соответствовать этому условию заменим на нули.

In [ ]:
df_estate['remodeled_year'] = df_estate.apply(lambda x: 0 if x.remodeled_year <= x.year_built else x.remodeled_year, axis=1)

Также создадим признак на основе remodeled_year - число лет, прошедших с реконструкции(age_remodeled). Обратим внимание, что если в признаке remodeled_year имеется 0, то тогда время реставрации считается с даты постройки дома:

In [ ]:
df_estate['age_remodeled'] = df_estate.apply(lambda x: (year_current - x.remodeled_year) if x.remodeled_year != 0 else (year_current - x.year_built), axis=1)

После этого можем удалить признаки year_built и remodeled_year.

In [ ]:
df_estate = df_estate.drop(['year_built', 'remodeled_year'], axis=1)

### **heating(отопление)**

In [ ]:
df_estate['heating'].isnull().sum()

В признаке 93105 пропущенных значений, так как значения текстовые, то приведём их для удобства в нижний регистр:

In [ ]:
df_estate['heating'] = df_estate['heating'].apply(lambda x: x.lower() if (type(x) is not float) and (x is not None) else x)
df_estate['heating'] = df_estate['heating'].apply(lambda x: np.nan if (x is None) | (x == 'none') | (x == '') else x)

Избавимся от ненужных символов, возьмём только первые два элемента:

In [ ]:
df_estate['heating'] = df_estate['heating'].apply(lambda x: x.replace(',', '') if type(x) is not float else x)
df_estate['heating'] = df_estate['heating'].apply(lambda x: x if (x is np.nan) | (type(x) is float) else ' '.join(x.split(' ')[:2]))

Вглянем на первые 50 самых распространённых значения:

In [ ]:
df_estate['heating'].value_counts()[:51]

Унифицируем данные, произведя необходимые замены, далее отберём первые 9 популярных значений, а остальные заменим на other. Пропущенные значения заменим самой распространённой категорией.

In [ ]:
df_estate['heating'] = df_estate['heating'].replace('central electric', 'electric').replace(' electric', 'electric').replace('no data', np.nan).replace('electric heat', 'electric')\
    .replace('natural gas', 'gas').replace(' gas', 'gas').replace('central gas', 'gas').replace('gas heat', 'gas').replace('electric gas', 'gas').replace('gas hot', 'gas')\
        .replace('gas -', 'gas').replace('propane', 'gas').replace(' heat', 'heat pump').replace('heating system', 'central').replace('central furnace', 'central')\
            .replace('central heating', 'central').replace('central heat', 'central').replace('heat pump(s)', 'heat pump').replace('radiant', 'central').replace('baseboard forced', 'baseboard')\
                .replace('baseboard hot', 'baseboard')

In [ ]:
# лист из топ 9 категорий
heating_list = list(df_estate['heating'].value_counts()[:9].index)

df_estate['heating'] = df_estate['heating'].apply(lambda x: x if x is np.nan else x if x in heating_list else 'other')
df_estate['heating'] = df_estate['heating'].fillna(df_estate['heating'].describe().top)

Итого получаем уменьшение категорий до 9:

- forced air(воздушное принудительное отопление)
- other(прочее)
- electric(электрическое)
- gas(газовое)
- central(центральное)
- central air(централье воздушное)
- heat pump(тепловые насосы)
- baseboard(тёплый пол)
- wall(тепловая стена)

In [ ]:
df_estate['heating'].value_counts()

### **cooling(охлаждение)**

В признаке типа охлаждения имеем 3092 пропуска:

In [ ]:
df_estate['cooling'].isnull().sum()

In [ ]:
df_estate['cooling'] = df_estate['cooling'].apply(lambda x: x.lower() if (x is not np.nan) & (x is not None) else x)
df_estate['cooling'] = df_estate['cooling'].apply(lambda x: np.nan if (x is None) | (x == 'none') | (x == '') else x)

In [ ]:
df_estate['cooling'].value_counts()[:40]

Произведём преобразование записей и разделим на категории:

In [ ]:
df_estate['cooling'] = df_estate['cooling'].replace('no data', np.nan).replace('none', np.nan)
df_estate['cooling'] = df_estate['cooling'].apply(lambda x: x if x is np.nan else ' '.join(x.split(' ')[:2]))
df_estate['cooling'] = df_estate['cooling'].apply(lambda x: x.replace(',', '') if type(x) is not float else x)
df_estate['cooling'] = df_estate['cooling'].replace('has cooling', 'cooling').replace('central a/c', 'central air')

Все значения, которые не войдут в топ-10 категорий обозначим как other, а пропущенные значение заменим самым распространённым значением.

In [ ]:
# топ лист
cooling_list = list(df_estate['cooling'].value_counts()[:10].index)
df_estate['cooling'] = df_estate['cooling'].apply(lambda x: x if x is np.nan else x if x in cooling_list else 'other')
df_estate['cooling'] = df_estate['cooling'].fillna(df_estate['cooling'].describe().top)

Итого в cooling(охлаждение) имеем категории:

- central(центральное)
- central air(центральное воздушное)
- other(прочее)
- cooling(охлаждение)
- central electric(центральное электрическое)
- central gas(центральное газ)
- wall(настенный)
- central heating(центральное отопление/охлаждение)
- cooling system(системы охлаждения)
- refrigeration(охлаждение)

In [ ]:
df_estate['cooling'].value_counts()

### **parking(паркинг)**

В признаке 3092(+150940) пропусков и очень много уникальных записей:

In [ ]:
df_estate['parking'].isnull().sum()

In [ ]:
df_estate['parking'].value_counts()[:50]

Произведём преобразование признака и унификацию категорий, далее оставим топ-15, остальное обозначим как other, пропуски заполним топовым значением.

In [ ]:
df_estate['parking'] = df_estate['parking'].apply(lambda x: x.lower() if (x is not np.nan) & (x is not None) else x)
df_estate['parking'] = df_estate['parking'].apply(lambda x: np.nan if (x is None) | (x == '') else x)
df_estate['parking'] = df_estate['parking'].replace('no data', np.nan).replace('none', np.nan)
df_estate['parking'] = df_estate['parking'].apply(lambda x: x if x is np.nan else x.replace('+', '').replace(',', '').split(' ')[0])
df_estate['parking'] = df_estate['parking'].replace('driveway', 'off').replace('garage', 'detached')
# топ лист
parking_list = list(df_estate['parking'].value_counts()[:15].index)

df_estate['parking'] = df_estate['parking'].apply(lambda x: x if x is np.nan else x if x in parking_list else 'other')
df_estate['parking'] = df_estate['parking'].fillna(df_estate['parking'].describe().top)

В признаке parking(паркинг) осталось 16 категорий:

- attached(пристроенный гараж)
- 0/1/2/3/4/5/6(количество мест)
- detached(отдельный гараж)
- carport(навес)
- off(место вне улицы)
- other(прочие)
- assigned(закреплённое место)
- parking(стоянка)
- electric(с электрическим приводом/воротами)

In [ ]:
df_estate['parking'].value_counts()

### **lotsize(дом с участком)**

In [ ]:
df_estate['lotsize'].isnull().sum()

In [ ]:
df_estate['lotsize'].value_counts()[:20]

В признаке 26667(+30552)пропущенных значений, но на самом деле их больше так как имеются неявные пропуски в виде "—", "No Data", "-- sqft lot". Видим, что значения занесены в разных единицах измерения - акры(acres) и квадратные фунты(sqft) - переведём всё к одному виду - sqft. Обратим внимание, что где-то имеется разделитель в виде точки, а где-то в виде запятой. После обработки заменим пропущенные значения медианным.

In [ ]:
def acr_change(acr):
    """ Переводит акры в квадратные фунты

    Args:
        acr (str): значение в акрах

    Returns:
        float: значение в квадратных фунтах
    """
    sqft = 43560 * float(acr)
    return sqft

def handler_lotsize(df):
    """ Обработчик признака lotsize

    Args:
        df (Series): необработанный признак

    Returns:
        float/str: обработанный признак
    """
    
    # заменяем пропуски на No и приводим разделитель к общему виду - точки
    df = df.replace('—', 'No').replace('No Data', 'No').replace('-- sqft lot', 'No').replace(',', '')
    
    if df != 'No':
        # приводим строку к нижнему регистру
        df = df.lower()
        # создаём лист со значениями 
        list_temp = df.split(' ')
        # если строка состоит из более чем одного значения, и второе значение 'acres' или 'acre', то переводим в кв фунты
        if (len(list_temp) > 1) and (df.split(' ')[1] == 'acres' or df.split(' ')[1] == 'acre'):
            df = acr_change(list_temp[0])
            return df
        else:
            df = float(list_temp[0])
            return df
    return df

In [ ]:
# меняем все пропущенные значения для удобства на No
df_estate['lotsize'] = df_estate['lotsize'].apply(lambda x: 'No' if (x is np.nan) | (x is None) | (x == '') else x)  
# применяем функцию обработчик
df_estate['lotsize'] = df_estate['lotsize'].apply(handler_lotsize)
# все No меняем на np.nan, отрицательные значения на 0
df_estate['lotsize'] = df_estate['lotsize'].apply(lambda x: np.nan if x == 'No' else 0 if x < 0 else x)
# пропуски заменяем на медианное значение
df_estate['lotsize'] = df_estate['lotsize'].fillna(df_estate['lotsize'].describe().median())

### **price_sqft(цена за квадратный фунт)**

In [ ]:
df_estate['price_sqft'].isnull().sum()

In [ ]:
df_estate['price_sqft'].value_counts()[:20]

В признаке имеется явный 41964 (+4304) пропуск значений и не явный в виде записей 'No Data', 'No Info', которые заменим на np.nan.

In [ ]:
df_estate['price_sqft'] = df_estate['price_sqft'].apply(lambda x: np.nan if (x == 'No Data') | (x == 'No Info') | (x is None) | (x == '') else x)
df_estate['price_sqft'] = df_estate['price_sqft'].apply(lambda x: x if x is np.nan else float(x.replace('$', '').replace('/', ' ').replace(',', '').split(' ')[0]))
# пропуски заменяем на медианное значение
df_estate['price_sqft'] = df_estate['price_sqft'].fillna(df_estate['price_sqft'].median())

После того как разобрались с блоком **homeFacts_new(сведения о доме)** данный признак можно удалить:

In [ ]:
df_estate = df_estate.drop(['homeFacts_new'], axis=1)
df_estate = df_estate.rename(columns={'street_new':'street', 'sqft_new':'sqft', 'stories_new':'stories', 'beds_new':'beds'})

### **schools(школы)**

Признак schools(школы) по структуре похож на признак honeFacts(сведения о доме) - формат json файла с вложенными словарями и списками в списке содержатся следующие данные:

- rating(рейтинг школ)
- data, содержащий несколько сведений:
    - Distance(дистанции до школ, в милях)
    - Grades(уровень школы - младшая/средняя/старшая/и тд)
    - name(наименование школы)

Нас будет интересовать только rating(рейтинг школ) и Distance(дистанции до школ, в милях), остальные признаки упустим. Вначале преобразуем строковое значение в словварь и вытащим необходимые данные, на основе которых создадим два новых признака.

In [ ]:
df_estate.iloc[1:2]

In [ ]:
df_estate['rating'] = df_estate['schools'].apply(lambda x: ast.literal_eval(x)[0]['rating'])
df_estate['distance'] = df_estate['schools'].apply(lambda x: ast.literal_eval(x)[0]['data']['Distance'])
#df_estate['grades'] = df_estate['schools'].apply(lambda x: ast.literal_eval(x)[0]['data']['Grades'])
df_estate = df_estate.drop(['schools'], axis=1)

### **rating(рейтинг школ)**

Каждое значение признака rating(рейтинг школ) представлен в виде списка с рейтингами школ,, список может быть пустым либо содержать 'NR'(нет рейтинга), 'NA'(нет данных). Мы создадим обобщающий признак - среднее значение рейтинга школ в данном районе - rating_mean. 

In [ ]:
df_estate['rating'].isnull().sum()

In [ ]:
def handler_rating(df):
    """ Обработчик признака rating

    Args:
        df (Series): необработанный признак

    Returns:
        float/np.nan: обработанный признак
    """
    temp_list = []
    list_rating = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
    
    if df == []:
        return np.nan
    else:
        for i in df:
            i = i.split('/')[0]
            if i in list_rating:
                temp_list.append(float(i))
        rating_mean = round(np.mean(temp_list), 2)
        return rating_mean   

In [ ]:
df_estate['rating_mean'] = df_estate['rating'].apply(handler_rating)
df_estate['rating_mean'] = df_estate['rating_mean'].fillna(round(df_estate['rating_mean'].mean(), 2))
df_estate = df_estate.drop(['rating'], axis=1)                                                           

### **disance(растояние до школ)**

В данном признаке, представленном в виде списка из расстояний до школ мы создадим два признака - среднее расстояние до школы(distance_mean) и количество школ на районе(schools_count).

In [ ]:
df_estate['distance'].isnull().sum()

In [ ]:
def handler_distance(df):
    """ Обработчик признака distance

    Args:
        df (Serise): необработанный признак

    Returns:
        list: обработанные признаки - средняя дистанция и количество школ
    """
    temp_list = []
    
    for i in df:
        i = i.split('mi')[0]
        temp_list.append(float(i))
    dist_mean = round(np.mean(temp_list), 2)
    schools_count = len(temp_list)
    return [dist_mean, schools_count]

In [ ]:
df_estate['distance_mean'] = df_estate['distance'].apply(lambda x: handler_distance(x)[0])
df_estate['schools_count'] = df_estate['distance'].apply(lambda x: handler_distance(x)[1])
df_estate['schools_count'] = df_estate['schools_count'].apply(lambda x: df_estate['schools_count'].median() if x==0 else x)
df_estate['distance_mean'] = df_estate['distance_mean'].fillna(round(df_estate['distance_mean'].mean(), 2))                                                 
df_estate = df_estate.drop(['distance'], axis=1)

In [ ]:
df_estate.shape[0]

Итак в очищенном датасете осталось 346004 наблюдения, взглянем ещё раз на целевую переменную, а далее попытаемся найти зависимости в признаках и сформировать гипотезы/выводы.

In [ ]:
fig = px.histogram(
    data_frame=df_estate,
    x='target_clean',    
    title='Distribution of the target_clean',
    text_auto=True,    
    height=800,    
    width=1500,
    marginal='box'     
)
fig.show()

In [ ]:
# ассиметрия 
print(df_estate['target_clean'].skew())

Распределение целевой переменной стало иметь визуально читаемый вид, но признак ассиметричный - коэффициент ассиметрии положительный(2.51), что говорит о смещении вправо - на графике имеем длинный правый хвост - в котором имеются наблюдения с большим диапазоном разброса цен:

- Медианное значение увеличилось до $327018
- Среднее - $481141
- Минимальное - $34700
- Максимальное - $3145000

### **status - target_clean**

In [ ]:
fig = px.histogram(
    data_frame=df_estate.groupby(['status'], as_index=False)['target_clean'].count(),
    x='status',
    y= ['target_clean'],
    color='status',
    height=500, 
    width=1000, 
    title='Distribution of the attribute status',
    text_auto=True   
)

fig.show()

In [ ]:
fig = px.bar(
    data_frame= df_estate.groupby(['status'], as_index=False)['target_clean'].median(),
    x='status',
    y='target_clean',
    color='status',
    text='target_clean', 
    orientation='v',
    title='Median value of the price from the status',
    height=500, 
    width=1000   
)

fig.show()

In [ ]:
fig = px.bar(
    data_frame= df_estate.groupby(['status'], as_index=False)['distance_mean'].median(),
    x='status',
    y='distance_mean',
    color='status',
    text='distance_mean', 
    orientation='v',
    title='Average distance to school by status',
    height=500, 
    width=1000   
)

fig.show()

В данных основная масса объявлений это со статусом **for sale**(225466), наименьшая группа **pending**(4385). 

Медианная стоимость недвижимости в зависимости от статуса разнится - наибольшая зафиксирована в категории **new construction**(449900), что вполне объяснимо - новое, более современное жильё и строится в более перспективных районах города. Стоит отметить и категории **foreclosure** - недвижимость должников для продажи ниже рынка(229000) и категории **pending** - возможно недвижимость в непристижных районах/либо далеко от инфраструктцры/школ - на графике видно, что среднее расстояние до школы больше(1,99 мили).

Ниже преведены для подтверждения статистические тесты - так, например для статуса "active", видим, что нет статистической разницы между "active" и "for sale" - но с другими статусами разница имеется.

In [ ]:
for_sale = df_estate[df_estate['status']=='for sale']['target_clean'].values
active = df_estate[df_estate['status']=='active']['target_clean'].values
foreclosure = df_estate[df_estate['status']=='foreclosure']['target_clean'].values
new_construction = df_estate[df_estate['status']=='new construction']['target_clean'].values
pending = df_estate[df_estate['status']=='pending']['target_clean'].values
other = df_estate[df_estate['status']=='other']['target_clean'].values

In [ ]:

_, p = f_oneway(for_sale, active, foreclosure, new_construction, pending, other)

H0 = 'Нет значимой разницы между ценой недвижимости с разными статусами продажи'
H1 = 'Есть значимая разница между ценой недвижимости с разными статусами продажи'

alpha = 0.05

if p > alpha:
    print(f'{p} > {alpha} мы не можем отвергнуть нулевую гипотезу. {H0}')
else:
    print(f'{p} <= {alpha} мы отвергаем нулевую гипотезу. {H1}')
    

In [ ]:
#perform Kruskal-Wallis Test 
_, p = stats.kruskal(for_sale, active, foreclosure, new_construction, pending, other)

alpha = 0.05

if p > alpha:
    print(f'{p} > {alpha} мы не можем отвергнуть нулевую гипотезу. {H0}')
else:
    print(f'{p} <= {alpha} мы отвергаем нулевую гипотезу. {H1}')

In [ ]:
import scikit_posthocs as sp
data = [for_sale, active, foreclosure, new_construction, pending, other]
sp.posthoc_dunn (data, p_adjust = 'bonferroni')[sp.posthoc_dunn (data, p_adjust = 'bonferroni') < 0.05]

### **propertyType - target_clean**

In [ ]:
fig = px.histogram(
    data_frame=df_estate.groupby(['propertyType'], as_index=False)['target_clean'].count().sort_values(by=['target_clean'], ascending=False),
    x='propertyType',
    y= ['target_clean'],
    color='propertyType',
    height=500, 
    width=1000, 
    title='Distribution of the attribute propertyType',
    text_auto=True   
)

fig.show()

In [ ]:
fig = px.bar(
    data_frame= df_estate.groupby(['propertyType'], as_index=False)['target_clean'].median().sort_values(by=['target_clean'], ascending=False),
    x='propertyType',
    y='target_clean',
    color='propertyType',
    text='target_clean', 
    orientation='v',
    title='Median value of the price from the propertyType',
    height=500, 
    width=1000   
)

fig.show()

Более 2/3 объявлений о продаже недвижимости приходится на категорию **single family**(на одну семью), также видим, что данный признак существенно влияет на цену недвижимости, так для земельных участков(**land**) она наименьшая - $150000.

### **state - target_clean**

Рассмотрим на распределение объявлений в штатах и зависимость целевой переменной от данного признака.

In [ ]:
fig = px.histogram(
    data_frame=df_estate.groupby(['state'], as_index=False)['target_clean'].count().sort_values(by=['target_clean'], ascending=False),
    x='state',
    y= ['target_clean'],
    color='state',
    height=500, 
    width=1000, 
    title='Distribution of the attribute propertyType',
    text_auto=True   
)

fig.show()

Отобразим объекты на географической карте:

In [ ]:
def to_int_size(value):
    try:
        return np.log10(int(value))
    except:
        return np.log10(int(value.split('[')[0]))

fig = go.Figure(go.Scattermapbox(lat=df_estate['latitude'], lon=df_estate['longitude'], text=df_estate[['city', 'age']], name='Estate on a geographical map', 
                                 marker=dict(size=df_estate['target_clean'].map(to_int_size), colorbar=dict(title="Age"), color=df_estate['age'])))
# центрирование карты на Нью-Йорк
capital = df_estate[df_estate['city']=='New York']
map_center = go.layout.mapbox.Center(lat=capital['latitude'].values[0], lon=capital['longitude'].values[0])
# Аналог с помощью словаря
#map_center =                   dict(lat=capital['geo_lat'].values[0], lon=capital['geo_lon'].values[0])

fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=map_center, zoom=3))
fig.show()

In [ ]:
fig = px.bar(
    data_frame= df_estate.groupby(['state'], as_index=False)['target_clean'].median().sort_values(by=['target_clean'], ascending=False),
    x='state',
    y='target_clean',
    color='state',
    text='target_clean', 
    orientation='v',
    title='Median value of the price from the state',
    height=500, 
    width=1000   
)

fig.show()

In [ ]:
treemap_data = df_estate.groupby(
    by='city',
    as_index=False
)[['target_clean']].median().sort_values(by=['target_clean'], ascending=False)[:101]

#строим график
fig = px.treemap(
    data_frame=treemap_data, #DataFrame
    path=['city'], #категориальный признак, для которого строится график
    values='target_clean', #параметр, который сравнивается
    height=500, #высота
    width=1500, #ширина
    title='Median price by city-100' #заголовок    
)

#отображаем график
fig.show()

Объявления представлены из 39 разных штатов(из 50), болше всего предложиний из штата Флорида(FL) - более 103 тыс. и штата Техас(TX) - более 81 тыс.

Что касаемое медианной цены недвижимости, то в топ-лидерах штаты:

- Нью-Йорк(NY) - $726850
- Массачусетс(MA) - $674700
- Калифорния(CA) - $659000
- Округ Колумбия(DC) - $629000

Наименьшая стоимость:

- OT - $50000
- Алабама(AL) - $72000
- Монтана(MT) - $82900
- Оклахома(OK) - $114750

Предполагаем о влиянии данного признака на цену недвижимости. 

Также стоит отметить топ-город по цене недвижимости:

- Saratoga(штат Калифорния) - $2998000
- Los Altos Hills(штат Калифорния) - $2980000
- New Richmond(штат Огайо) - $2980000
- Highway 30(штат Оклахома) - $2900000
- Bow Mar(штат Колорадо) - $2750000
- Rancho Park(штат Калифорния) - $2750000
- Santa Monica(штат Калифорния) - $2695000
- Sunset Beach(штат Серерная Каролина) - $2595000

Наименьшая стоимость:

- Ford Hancock(штат Техас) - $35000
- Wayland(штат Миссури) - $35200
- Mem () - $35900
- Minnie(штат Флорида) - $37000
- Shillington(штат Пенсильвания) - $38566

### **beds - target_clean**

In [ ]:
fig = px.histogram(
    data_frame=df_estate,
    x='beds',    
    title='Distribution of the beds',
    text_auto=True,    
    height=500,    
    width=1000      
)
fig.show()

In [ ]:
fig = px.scatter(
    data_frame=df_estate.groupby(['beds'], as_index=False)['target_clean'].median(),
    x='beds',   
    y='target_clean',
    color='beds',
    title='Scatterplot between beds and target_clean(median)',       
    height=500,    
    width=1000,
    size='target_clean'       
)
fig.show()

Видим зависимость цены недвижимости от числа комнат - при чём до трёх включительно разница небольшая, после трёх видно ускорение в увеличении стоимости: особенно скачок после 4 комант, после 10 комнат. Опять же с 7 до 10 комнат разница несущественна - возможна связано, с небольшой разницей в общей площади дома и более меньшими площадями самих комнат. Топовое количество комнат - 3(более 198 тыс. объектов). Интересны объекты с 1/9/13 комнатами, где стоимость ниже аналогичных объектов с меньшим количеством комнат.

### **baths - target_clean**

In [ ]:
fig = px.histogram(
    data_frame=df_estate,
    x='baths',    
    title='Distribution of the baths',
    text_auto=True,    
    height=500,    
    width=1000,
    nbins=20     
)
fig.show()

In [ ]:
fig = px.scatter(
    data_frame=df_estate.groupby(['baths'], as_index=False)['target_clean'].median(),
    x='baths',   
    y='target_clean',
    color='baths',
    title='Scatterplot between baths and target_clean(median)',       
    height=500,    
    width=1000,
    size='target_clean'       
)
fig.show()

Картинка неоднозначная - да, видим, что при увеличении количества ванных комнат растёт и медианная стоимость недвижимости, но интересны некоторые значения, которые выпадают из общего тренда:

- участок с 10 по 12 включительно цена наоборот снижается - в том числе за счёт того, что снижается средняя площадь недвижимости как увидим позже
- значения baths с 17/19/20 - через чур низкие - объясняется тем, что данные объекты расположены в штатах с более низкой медианной стоимостью недвижимости(Техас(TX), Флорида(FL)). Представлено немного объектов
- значения 5.2, 4.75, 3.2 - через чур высокие - недвижимость из априори дорогих штатов. Представлено немного объектов
- значения 6.5/8.5 - ниже - из-за того, что основная масса предложений в данной категории из штата Флорида(FL), а это недвижимости типа single family. Если посмотреть на медианную площадь данной недвижимости в штате Флорида и в других штатах, то мы увидим, что во Флориде она существенно меньше(3544 против 6590 sqft). Представлено немного объектов

Основная доля объектов приходится с количеством ванных комнат от 2 до 4(медианное значение 2.25). Зависимость прослеживается.

In [ ]:
print(f'Медианная площадь дома 6.5/8.5 в других штатах - {df_estate[(df_estate["state"]!="FL") & ((df_estate["propertyType"]=="single family")) & ((df_estate["baths"]==8.50) | (df_estate["baths"]==6.50))]["sqft"].median()}')

print(f'Медианная площадь дома 6.5/8.5 в штатае FL - {df_estate[(df_estate["state"] == "FL") & ((df_estate["propertyType"]=="single family")) & ((df_estate["baths"]==8.50)|(df_estate["baths"]==6.50))]["sqft"].median()}')

### **sqft - target_clean**

Признак sqft имеет большой разброс(квадратичное отклонение 772338) - от 0 до 456602500 квадратных фунтов, при этом медианное значение равняется 1800. Пока никаких манипуляций с ним производить не будем, но учтём этот момент на будущее.

In [ ]:
df_estate['sqft'].describe()

In [ ]:
fig = px.scatter(
    data_frame=df_estate.groupby(['age'], as_index=False)['sqft'].median(),
    y='sqft',   
    x='age',
    color='sqft',
    title='Scatterplot between age and sqft(median)',       
    height=500,    
    width=1000,
    #size='sqft'       
)
fig.show()

Интересную информацию даёт график зависимости медианной площади недвижимости в зависимости от возраста:

- старые дома(более 100-120 лет - старше 1900 г) имеют более большой диапазон в площадях домов, они разнятся от года к году, много домов большой площади, далее они приходят к более единой медианной площади разнявшихся несильно. Отметим, что если необходим дом с большой площадью, то стоит его подыскивать именно в этой категории.
- период 90-100 лет(1922-1932 гг) - замечено увеличение площадей дома(в среднем с 1560 до 1800) - как раз в Америке совпал данный период с бурным экономическим ростом, закончившись великой депрессией 1929г.
- период 80-90 лет(1932-1942 гг) - снижение медианных площадей(до 1344) опять же вызванных экономическим кризисом
- период 58-80 (1942-1964 гг) - бурный рост площадей до 1700 совпавшим с экономическим ростом
- периоды 49-58(1964-1980 гг) - снижение до 1426(1973 г. - нефтяной кризис), 1975-1981(снижение до 1477 с ростом в 1976)
- период 20-49(1981-2000гг) - бурный рост до 2130
- период 0-20 (2000-2022гг) - наблюдалось снижение(до 1700) на фоне кризиса в Америке в нулевых, далее взлёт до 2506 и коррекция к 2018 с тенденцией на уменьшение площади


In [ ]:
fig = px.scatter(
    data_frame=df_estate.groupby(['baths'], as_index=False)['sqft'].median(),
    y='sqft',   
    x='baths',
    color='baths',
    title='Scatterplot between baths and sqft(median)',       
    height=500,    
    width=1000,
    size='sqft'       
)
fig.show()

Ранее мы рассматривали зависимость целевой переменной от количества ванных комнат и выявили объекты, у которых стоимость недвижимости снижалось после достижения ванных комнат 9 и  до 15, и мы выдвигали гипотезу, что с ростом ванных комнат растёт и цена недвижимости, но при достижении числа 9(и вплоть до 15)начинается снижение медианной площади недвижимости - график подтвердил наши догадки.


In [ ]:
fig = px.bar(
    data_frame= df_estate.groupby(['propertyType'], as_index=False)['sqft'].median().sort_values(by=['sqft'], ascending=False),
    x='propertyType',
    y='sqft',
    color='propertyType',
    text='sqft', 
    orientation='v',
    title='Median value of the sqft from the propertyType',
    height=500, 
    width=1000   
)

fig.show()

Ожидаемые ТОП типов недвижимости по площадям:

- multi family - 2200
- traditional - 2144
- single detached - 2051.5
- single family - 1911

Наимение это condo(1195), coop(1200), high rise(1257)

### **stories -target_clean**

In [ ]:
fig = px.histogram(
    data_frame=df_estate,
    x='stories',    
    title='Distribution of the stories',
    text_auto=True,    
    height=500,    
    width=1000,
    nbins=40     
)
fig.show()

In [ ]:
fig = px.scatter(
    data_frame=df_estate.groupby(['stories'], as_index=False)['target_clean'].median(),
    x='stories',   
    y='target_clean',
    color='stories',
    title='Scatterplot between stories and target_clean(median)',       
    height=500,    
    width=1000,
    size='target_clean'       
)
fig.show()

Основная часть недвижимости расположена на 2-3 этажах, медианное значение - 1 этаж, что вполне объяснимо так как это частные дома. Имеется плавная прямая зависимость стоимости от этажности строения.

### **heating-target_clean**

In [ ]:
fig = px.histogram(
    data_frame=df_estate,
    x='heating',    
    title='Distribution of the heating',
    text_auto=True,    
    height=500,    
    width=1000        
)
fig.show()

In [ ]:
fig = px.bar(
    data_frame= df_estate.groupby(['heating'], as_index=False)['target_clean'].median().sort_values(by=['target_clean'], ascending=False),
    x='heating',
    y='target_clean',
    color='heating',
    text='target_clean', 
    orientation='v',
    title='Median value of the target_clean from the heating',
    height=500, 
    width=1000   
)

fig.show()

Преобладающая часть недвижимость с отоплением типа forced air - медианная стоимость такой недвижимости - $335000, самая дорогая недвижимость с central air/central - $390000/$357900, а с электрическим отоплением(electric) наименьшая - $270000.

### **cooling - target_clean**

In [ ]:
fig = px.histogram(
    data_frame=df_estate,
    x='cooling',    
    title='Distribution of the cooling',
    text_auto=True,    
    height=500,    
    width=1000        
)
fig.show()

In [ ]:
fig = px.bar(
    data_frame= df_estate.groupby(['cooling'], as_index=False)['target_clean'].median().sort_values(by=['target_clean'], ascending=False),
    x='cooling',
    y='target_clean',
    color='cooling',
    text='target_clean', 
    orientation='v',
    title='Median value of the target_clean from the cooling',
    height=500, 
    width=1000   
)

fig.show()

### **parking - target_clean**

In [ ]:
fig = px.histogram(
    data_frame=df_estate,
    x='parking',    
    title='Distribution of the parking',
    text_auto=True,    
    height=500,    
    width=1000        
)
fig.show()

In [ ]:
fig = px.bar(
    data_frame= df_estate.groupby(['parking'], as_index=False)['target_clean'].median().sort_values(by=['target_clean'], ascending=False),
    x='parking',
    y='target_clean',
    color='parking',
    text='target_clean', 
    orientation='v',
    title='Median value of the target_clean from the parking',
    height=500, 
    width=1000   
)

fig.show()

Более 2/3 недвижимости в признаке parking представлены attached(пристроенный гараж) с медианной стоимостью $32500. Интересно, что имеется недвижимость без парковочного места(0) и её стоимость $1100000, что странно, но это объясняется тем, что данная недвижимость представлена из престижного штата Калифорния(CA). 

In [ ]:
df_estate.info()

### **rating_mean/schools_count - target_clean** 

In [ ]:
fig = px.histogram(
    data_frame=df_estate,
    x='rating_mean',    
    title='Distribution of the rating_mean',
    text_auto=True,    
    height=500,    
    width=1000,
    nbins=40      
)
fig.show()

In [ ]:
fig = px.histogram(
    data_frame=df_estate,
    x='schools_count',    
    title='Distribution of the schools_count',
    text_auto=True,    
    height=500,    
    width=1000,
    nbins=40        
)
fig.show()

In [ ]:
fig = px.scatter(
    data_frame=df_estate.groupby(['rating_mean'], as_index=False)['target_clean'].median().sort_values(by=['target_clean'], ascending=False),
    x='rating_mean',   
    y='target_clean',
    color='rating_mean',
    title='Scatterplot between rating_mean and target_clean(median)',       
    height=500,    
    width=1000,
    size='target_clean'       
)
fig.show()

In [ ]:
fig = px.bar(
    data_frame= df_estate.groupby(['state'], as_index=False)['rating_mean'].median().sort_values(by=['rating_mean'], ascending=False),
    x='state',
    y='rating_mean',
    color='state',
    text='rating_mean', 
    orientation='v',
    title='Median value of the rating_mean from the state',
    height=500, 
    width=1000   
)

fig.show()

In [ ]:
fig = px.scatter(
    data_frame=df_estate.groupby(['schools_count'], as_index=False)['target_clean'].median().sort_values(by=['target_clean'], ascending=False),
    x='schools_count',   
    y='target_clean',
    color='schools_count',
    title='Scatterplot between schools_count and target_clean(median)',       
    height=500,    
    width=1000,
    size='target_clean'       
)
fig.show()

In [ ]:
df_estate['rating_mean'].describe()

In [ ]:
df_estate['distance_mean'].describe()

Средний рейтинг школ - 5.17, медианное количество школ около объекта недвижимости - 3, расстояние до которых 1.75 мили. От рейтинга школы зависимость целевой переменной прослеживается, но слабая, а вот количество школ в общем не влияет на цену недвижимости, но имеются отдельные выбросы не входящий в общий тренд.

### **age - target_clean**

In [ ]:
fig = px.histogram(
    data_frame=df_estate,
    x='age',    
    title='Distribution of the age',
    text_auto=True,    
    height=500,    
    width=1000,
    nbins=50,
    marginal='box'     
)
fig.show()

In [ ]:
fig = px.scatter(
    data_frame=df_estate.groupby(['age'], as_index=False)['target_clean'].median(),
    x='age',   
    y='target_clean',
    color='age',
    title='Scatterplot between age and target_clean(median)',       
    height=500,    
    width=1000,
    #size='target_clean'       
)
fig.show()

По возрасту недвижимости преобладают объекты 30-39 лет(медианный возраст - 38 лет). В среднем с увеличением возраста недвижимости прослеживается тенденция к уменьшению её стоимости, но довоенная недвижимость(80-87 лет) также держится в цене. Вообщем, что касаемо объектов старше 100 лет, то их можно разделить на два класса - те которые имеют цену ниже рыночной - возможно это объекты требующие вложений/реставрации/расположенных в непристижных районах и второй класс - те, которые стоят существенно дороже - какие-то экслюзивные варианты, также имеющие большую площадь/либо большой земельный участок/лучшую локацию.

Опишем типичный объект из нашего датасета: этот объект является одноэтажным(stories) домом для одной семьи(single family) площадью 1800 квадратных фунтов(sqft) и площадью участка(lotsize) 10890 квадратных фунтов, имеется пристроенный гараж(attached), принудительное воздушное отопление(forced air), 3 комнаты(beds) и 2.5 ванны комнаты(baths), возраст дома 38 лет, рядом(около 1.75 миль) находятся 3 школы(schools_count) со средним рейтингом (rating_mean) равным 5.17. Объект в статусе для продажи(for sale) и оценивается около $327018.

### Используем автоматическую визуализацию и представление датасета с помощью D-Tale

In [ ]:
# Интерактивный аудит данных

dtale_app.USE_COLAB = True 
d = dtale.show(df_estate)
print(f"Интерактивный отчет доступен по ссылке: {d._main_url}")

### **Отбор признаков**

В нашем датасете сформировано 24 признака, 9 из которых имеют тип object, закодируем их, далее полученные признаки будем приводить к одному виду - нормировать/стандартизировать т.к. числовые величины, которыми описываются признаки имеют разный порядок и более большие по модулю сбивают модель в свою сторону. Для начала построим матрицу корреляции для обнаружения линейной связи между целевым признаком и предикторами, а также признаки, которые сильно закоррелированные между собой:

In [ ]:
df_estate.info()

In [ ]:
# строим матрицу корреляции
fig, axes = plt.subplots(figsize=(24, 10))
sns.heatmap(round(df_estate.corr(method='spearman', numeric_only=True), 2), annot=True)

Видим корреляцию целевого признака с:
- price_sqft - 0.66
- sqft - 0.48
- baths - 0.38
- rating_mean - 0.33
- beds - 0.24

заметны сильноскореллированные признаки, такие как, age/age_remodeled(0.83), zip_code/longitude(-0.93) - часть из них придётся убрать. Остаётся вопрос о спорном признаке price_sqft - через который как бы происходит утечка информации о целевом признаке, удалим его. 

В наших данных имеются несколько признаков ниже, которые сообщают нам одну и ту же информацию, а именно о локации объекта недвижимости - данный фактор существенен для определения стоимости недвижимости:

- city
- state
- zipcode
- county
- latitude
- longitude
- street

Все признаки типа object, кроме zipcode(переведён нами в числовой), latitude, longitude - оставим эти три признака опеределяющих местоположение, остальные удалим. Также стоит заметить, что признак street использовать не получиться из-за большого количества уникальных значений и при кодировании получим очень много признаков.

Подготовим данные: удалим часть предикторов, необходимые закодируем, далее выборку разделим на тренировочну и тестовую в соотношении 70/30, далее нормализуем данные.

In [ ]:
X = df_estate.drop(['city', 'street', 'state', 'county', 'age_remodeled', 'price_sqft', 'target_clean', 'target_clean_log'], axis=1)
y = df_estate['target_clean']
object_columns = [s for s in X.columns if X[s].dtypes == 'object']
X = pd.get_dummies(X, columns=object_columns, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, 
    test_size=0.30, 
    random_state=RANDOM_SEED,
    shuffle=True
)

# инициализируем нормализатор RobustScaler
r_scaler = preprocessing.RobustScaler()

# кодируем исходный датасет
X_train_scal = r_scaler.fit_transform(X_train)

# Преобразуем промежуточный датасет в полноценный датафрейм для визуализации
X_train_scal = pd.DataFrame(X_train_scal, columns=X_train.columns)
X_test_scal = pd.DataFrame(r_scaler.transform(X_test), columns=X_test.columns)

In [ ]:
X_train_scal.shape

## 4. Предварительный анализ моделей и Baseline

### **Линейная регрессия**

Для начала сформируем базовую модель, которую будем улучшать базовая модель будет строиться на всех признаках(59), далее производим предсказание - на тренировочных и на тестовых данных и проверяем метрики. 

В качестве метрик используем три метрики - MAE(используем больше для себя - для понимания в абсолютных величинах на сколько в среднем ошибается модель, данную величину легко сравнить с медианной стоимостью недвижимости), MSE и R2 для сравнения моделей между собой.

In [ ]:
def metrics_func(y_train, y_train_pred, y_test, y_test_predict, model_name="Unnamed Model"):
    """
    Вычисляет метрики, красиво пишет их в лог и сохраняет в глобальную таблицу экспериментов.
    """
    
    # 1. Вычисляем метрики
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_test_predict)
    
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_predict)
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_predict)
    
    # 2. Формируем красивое сообщение для лога
    # :<20 - выравнивание влево, ширина 20
    # :>15,.2f - выравнивание вправо, ширина 15, разделитель тысяч, 2 знака после запятой
    log_msg = (
        f"\n{'='*65}\n"
        f"MODEL: {model_name}\n"
        f"{'-'*65}\n"
        f"{'Metric':<10} | {'Train':>20} | {'Test':>20}\n"
        f"{'-'*65}\n"
        f"{'MAE':<10} | {mae_train:>20,.2f} | {mae_test:>20,.2f}\n"
        f"{'MSE':<10} | {mse_train:>20,.0f} | {mse_test:>20,.0f}\n"
        f"{'R2':<10} | {r2_train:>20.4f} | {r2_test:>20.4f}\n"
        f"{'='*65}\n"
    )
    
    # Вывод в ноутбук и файл
    print(log_msg)
    logging.info(log_msg)
    
    # 3. Сохраняем в глобальный список (для итоговой таблицы)
    EXPERIMENTS_DATA.append({
        'Model': model_name,
        'R2 Score (Train)': round(r2_train, 3),
        'R2 Score (Test)': round(r2_test, 3),
        'MAE (Train)': round(mae_train, 0),
        'MAE (Test)': round(mae_test, 0),
        'MSE (Train)': round(mse_train, 0),
        'MSE (Test)': round(mse_test, 0)
    })

In [ ]:
model_base = LinearRegression()
model_base.fit(X_train_scal, y_train)
y_train_predict = model_base.predict(X_train_scal)
y_test_predict = model_base.predict(X_test_scal)
metrics_func(y_train, y_train_predict, y_test, y_test_predict, model_name="LinearRegression (baseline)")

Базовая модель показала не лучшие метрики, т.к. абсолютная ошибка в размере более $261786 является очень большой. 

Коэффицент детерминации сильно различается для тренировочных и тестовых данных, что говорит о том, что модель не уловила зависимости, а скорее всего звисимость сложнее (нелинейная). 
Поробуем обучить модель на 25 отобранных лучших признаках - для этого используем SelectKBest:

In [ ]:
selector = feature_selection.SelectKBest(feature_selection.f_regression, k=25)
selector.fit(X_train_scal, y_train)
best_features = selector.get_feature_names_out()
best_features

Интересно отметить, что в список топ-25 не вошли такие признаки, как sqft(площадь дома), который указывает на общий размер недвижимости и zipcode(индекс), который указывает на локацию недвижимости, а она влияет непостедственно на его цену. Без данных признаков модель скорее всего будет несостоятельна. 

Также стоит предположить, что данная проблема возникла из-за того, что в алгоритме **SelectBest** используется корреляция Пирсона, что в данной задаче недопустимо из-за того, что целевой признак не является нормально распределённым и необходимо исследовать линейные связи между предикторами и целевой переменной опираясь на корреляцию Спирмена. 

По корреляции Спирмена между sqft и target_clean составляет 0.48(можно увидеть на графике корреляции выше), но как будет продемонстрировано ниже, если использовать корреляцию Пирсона, то значение не будет превышать 0.029, поэтому он не отобран в ТОП-25. 
Что касаемо признака zipcode, то и корреляция Спирмена невысока - 0.01 - здесь дело в том, что данный предиктор хоть и представлен в числовом виде является категориальной переменной и необходимо её использовать именно в виде списка категорий индексов предварительно произведя их разбивку. На данном этапе будем выполнять построение моеделей согласно отобранным признакам.

In [ ]:
# корреляция Пирсона между sqft/target_clean
np.corrcoef(X_train_scal['sqft'], y_train)

Первой построим линейную регрессию.

In [ ]:
model_1 = LinearRegression()
model_1.fit(X_train_scal[best_features], y_train)
y_train_predict = model_1.predict(X_train_scal[best_features])
y_test_predict = model_1.predict(X_test_scal[best_features])
metrics_func(y_train, y_train_predict, y_test, y_test_predict, model_name="LinearRegression (best_features = 25)")

Модель, построенная на 25 лучших признаках показала стабильно метрику $R^2=0.24$, но улавливаемая закономерность линейной модели очень низкая. 


### DecisionTreeRegressor

Перейдём к модели деревьям решений.

In [ ]:
tree_model = tree.DecisionTreeRegressor(random_state=RANDOM_SEED)
tree_model.fit(X_train_scal[best_features], y_train)

y_train_pred = tree_model.predict(X_train_scal[best_features])
y_test_pred = tree_model.predict(X_test_scal[best_features])

metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name="DecisionTreeRegressor (Top-25 features)")

Данную модель построили на всех признаках, очевидно, что есть явное переобучение модели. Поэтому соответственно подберём оптимальные параметры глубины дерева.

In [ ]:
#from sklearn import metrics

max_depths = range(7, 20)

def tree_depths(X_train, X_test, y_train, y_test):
  R_2_train = []
  R_2_test = []
  for depth in max_depths:
    tree_model = tree.DecisionTreeRegressor(max_depth=depth, random_state=RANDOM_SEED).fit(X_train, y_train)
    y_train_pred = tree_model.predict(X_train)
    y_test_pred = tree_model.predict(X_test)
    R_2_tr = round(r2_score(y_train, y_train_pred), 2)
    R_2_tes = round(r2_score(y_test, y_test_pred), 2)
    R_2_train.append(R_2_tr)
    R_2_test.append(R_2_tes)

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=list(max_depths), y=R_2_train, name='Train'))
  fig.add_trace(go.Scatter(x=list(max_depths), y=R_2_test, name='Test'))
  fig.update_xaxes(title='Depth count')
  fig.update_yaxes(title='R_2')
  fig.update_layout(title='Dependence of R_2 on the number of trees', height=600, width=800)
  return fig.show()

tree_depths(X_train_scal, X_test_scal, y_train, y_test)

Выберем за оптимальную шлубину - 13 деревьев и построим модель:

In [ ]:
tree_model = tree.DecisionTreeRegressor(random_state=RANDOM_SEED, max_depth=13)
tree_model.fit(X_train_scal[best_features], y_train)

y_train_pred = tree_model.predict(X_train_scal[best_features])
y_test_pred = tree_model.predict(X_test_scal[best_features])

metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name="DecisionTreeRegressor (Top-13 features)")

Метрики вновь улучшились, $R^2 - 0.61/0.53$, но обратим внимание, что метрика на тренировочных данных лучше более, чем на 13%, что может говорить о переобучении модели и возможно необходимо снизить глубину.

### Ансамблевые методы

Первой ансамблевой моделью будет бэггинг и его разновидность - модель случайного леса.

In [ ]:
# бэггинг случайный лес
random_forest = ensemble.RandomForestRegressor(n_estimators=200,
                                               max_depth=16,                                               
                                               criterion='squared_error',
                                               random_state=RANDOM_SEED)

random_forest.fit(X_train_scal[best_features], y_train)

y_train_pred = random_forest.predict(X_train_scal[best_features])
y_test_pred = random_forest.predict(X_test_scal[best_features])

metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name="RandomForestRegressor (Top-25 features)")

Следующая модель градиентного бустинга:

In [ ]:
# Создаем объект класса градиентный бустинг
gb = GradientBoostingRegressor(
    max_depth=13, #максимальная глубина дерева
    n_estimators=500, #количество деревьев в ансамбле
    random_state=RANDOM_SEED, #датчик генератора случайных чисел
    learning_rate=0.01
)

# Обучаем модель
gb.fit(X_train_scal[best_features], y_train)

# Формируем предсказание для тестовой выборки

y_train_pred  = gb.predict(X_train_scal[best_features])
y_test_pred = gb.predict(X_test_scal[best_features])

metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name="GradientBoostingRegressor (Top-25 features)")

Ещё одна модель бустинга CatBoost:

In [ ]:
# модель бустинга CatBoost
catmodel = CatBoostRegressor(random_state=RANDOM_SEED, verbose=False)
catmodel.fit(X_train_scal[best_features], y_train)

y_train_pred  = catmodel.predict(X_train_scal[best_features])
y_test_pred = catmodel.predict(X_test_scal[best_features])
metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name="CatBoostRegressor (Top-25 features)")


Итого в фаворитах оказываются модели - случайный лес($R^2=0.69/0.60$), градиентный бустинг($R^2=0.74/0.63$) и CatBoost($R^2=0.61/0.6$). Но, вспомним о том, на каких признаках были построены данные модели - 25 лучших предикторов отобранных с помощью SelectBest. Как уже отмечалось ранее после отбора в топ-25 не вошёл такой признак как площадь недвижимости sqft, который по логике должен напрямую влиять на стоимость недвижимости. Воспользуемся CatBoost для построения модели на всех признаках, а далее отберём топ-предикторы согласно их весам.

In [ ]:
catmodel = CatBoostRegressor(random_state=RANDOM_SEED, verbose=False)
catmodel.fit(X_train_scal, y_train)

y_train_pred  = catmodel.predict(X_train_scal)
y_test_pred = catmodel.predict(X_test_scal)

metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name="CatBoostRegressor (All features)")

In [ ]:
# признаки и их веса вносимые в модель
best_feature_cat = pd.DataFrame({'feature_importance': catmodel.get_feature_importance(), 
              'feature_names':X_train_scal.columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)
# топ-11 признаков по весам
new_best = best_feature_cat['feature_names'].iloc[:11]
best_feature_cat[:11]

Как видим по степени значимости на втором месте площадь недвижимости(sqft). Далее построим отмеченные ранее модели на новых топ-11 признаках.

In [ ]:
catmodel = CatBoostRegressor(random_state=RANDOM_SEED, verbose=False)
catmodel.fit(X_train_scal[new_best], y_train)

y_train_pred  = catmodel.predict(X_train_scal[new_best])
y_test_pred = catmodel.predict(X_test_scal[new_best])

metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name="CatBoostRegressor (Top-11 features)")

In [ ]:
random_forest = ensemble.RandomForestRegressor(n_estimators=200,
                                               max_depth=13,                                               
                                               criterion='squared_error',
                                               random_state=RANDOM_SEED)

random_forest.fit(X_train_scal[new_best], y_train)

y_train_pred = random_forest.predict(X_train_scal[new_best])
y_test_pred = random_forest.predict(X_test_scal[new_best])

metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name="RandomForestRegressor (Top-11 features)")

In [ ]:
# визуализируем важные признаки в дереве решений
oo = pd.DataFrame([random_forest.feature_importances_], columns=X_train_scal[new_best].columns)
fig = px.bar( 
    x=list(oo.loc[0].sort_values(ascending=False).index),
    y=round(oo.loc[0].sort_values(ascending=False), 2),
    text_auto=True,
    title='ТОП-11 features for RandomForest',
    height=500, 
    width=1000,
    labels={'x':'feature_importances', 'y':'weight'}
       
)
fig.show()

In [ ]:
# Создаем объект класса градиентный бустинг
gb = GradientBoostingRegressor(
    max_depth=9, #максимальная глубина дерева
    n_estimators=500, #количество деревьев в ансамбле
    random_state=RANDOM_SEED, #датчик генератора случайных чисел
    learning_rate=0.01
)

# Обучаем модель
gb.fit(X_train_scal[new_best], y_train)

# Формируем предсказание для тестовой выборки

y_train_pred  = gb.predict(X_train_scal[new_best])
y_test_pred = gb.predict(X_test_scal[new_best])

metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name="GradientBoostingRegressor (Top-11 features)")

In [ ]:
# визуализируем важные признаки в градинетном бустинге
aa = pd.DataFrame([gb.feature_importances_], columns=X_train_scal[new_best].columns)
fig = px.bar( 
    x=list(aa.loc[0].sort_values(ascending=False).index),
    y=round(aa.loc[0].sort_values(ascending=False), 2),
    text_auto=True,
    title='ТОП-11 features for GradientBoosting',
    height=500, 
    width=1000,
    labels={'x':'feature_importances', 'y':'weight'}
       
)
fig.show()

In [ ]:
# Создание матриц наблюдений в формате DMatrix
dtrain = xgb.DMatrix(X_train_scal[new_best], label=y_train, feature_names=new_best.tolist())
dtest = xgb.DMatrix(X_test_scal[new_best], label=y_test, feature_names=new_best.tolist())

# Гиперпараметры модели
xgb_pars = {'min_child_weight': 20, 'eta': 0.1, 'colsample_bytree': 0.9, 
            'max_depth': 6, 'subsample': 0.9, 'lambda': 1, 'nthread': -1, 
            'booster' : 'gbtree', 'eval_metric': 'rmse', 'objective': 'reg:squarederror'
           }
# Тренировочная и валидационная выборка
watchlist = [(dtrain, 'train'), (dtest, 'valid')]
# Обучаем модель XGBoost
model_xgb = xgb.train(
    params=xgb_pars, #гиперпараметры модели
    dtrain=dtrain, #обучающая выборка
    num_boost_round=300, #количество моделей в ансамбле
    evals=watchlist, #выборки, на которых считается матрица
    early_stopping_rounds=20, #раняя остановка
    maximize=False, #смена поиска максимума на минимум
    verbose_eval=10 #шаг, через который происходит отображение метрик
)

y_train_predict = model_xgb.predict(dtrain)
y_test_predict = model_xgb.predict(dtest)
metrics_func(y_train, y_train_predict, y_test, y_test_predict, model_name="XGBRegressor (Top-11)")

In [ ]:
fig, ax = plt.subplots(figsize = (9,9))
xgb.plot_importance(model_xgb, ax = ax, height=0.5)

Первичный анализ моделей показал, что ключевыми признаками являются longitude, latitude, sqft, zipcode (если он закодирован), lotsize, age, rating_mean, distance_mean, baths, beds, stories. Линейные связи слабые, поэтому имеет смысл использовать ансамблевые методы.

## 5. Оценка построенных моделей

**Промежуточный вывод:**
 1.  Простые линейные модели показывают среднее качество ($R^2 \approx 0.55$), что говорит о сложной нелинейной природе данных.
 2.  Базовые ансамбли (RandomForest/CatBoost) "из коробки" дают неплохой результат ($R^2 \approx 0.75-0.78$).
 3.  **Проблема:** Текущий подход требует ручной обработки признаков перед каждой моделью, что неудобно и чревато ошибками.

**Дальнейшие шаги:** Для улучшения результата и создания надежного решения мы перейдем к использованию **Sklearn Pipelines**, добавим продвинутую обработку категорий (`TargetEncoder`) и проведем оптимизацию гиперпараметров.


**Анализ результатов базовых моделей:**
Мы протестировали простые подходы на очищенных данных.
1.  **Провал автоматического отбора признаков:** Модель `LinearRegression (Top-25)` показала худший результат ($R^2=0.26$) по сравнению с моделью на всех признаках ($R^2=0.54$).
*   *Причина:* Стандартные методы отбора (по корреляции) исключили географические признаки (`latitude`, `longitude`), так как их связь с ценой нелинейна. Это доказывает, что для недвижимости нельзя удалять гео-данные.
2.  **Ограниченность One-Hot Encoding:** Базовые ансамбли (RF, CatBoost) показали результат $R^2 \approx 0.60-0.75$. Они не смогли полностью раскрыть потенциал данных из-за высокой кардинальности признака `zipcode` (4000+ уникальных значений), который сложно эффективно закодировать стандартными методами.
3.  **Проблема:** Текущий подход требует ручной обработки признаков перед каждой моделью, что неудобно и чревато ошибками.

**Решение:** Для улучшения метрик мы переходим к использованию **Pipeline** с применением **Target Encoding** для почтовых индексов.


In [ ]:
# # Предварительно находим строки с аномально большими остатками (выбросы)
# # 1. Формируем X и y
# X = df_estate.drop(['city', 'street', 'state', 'county', 'age_remodeled', 'price_sqft', 'target_clean'], axis=1)
# y = df_estate['target_clean']

# # 2. Находим ВСЕ категориальные признаки (и object, и category)
# cols_to_encode = X.select_dtypes(include=['object', 'category']).columns

# # 3. Делаем One-Hot Encoding, dtype=int превращает True/False сразу в 1/0
# X = pd.get_dummies(X, columns=cols_to_encode, drop_first=True, dtype=int)

# # 4. Финальная проверка перед OLS (страховка), Оставляем только числа
# X = X.select_dtypes(include=['number'])

# # 5. Анализ стандартизированных остатков, Добавляем константу (интерцепт)
# X_const = sm.add_constant(X)

# estate_outlier = sm.OLS(y, X_const)
# result = estate_outlier.fit()

# influence = OLSInfluence(result)
# sresiduals = influence.resid_studentized_internal

# # Вывод результатов
# print("Максимальный и минимальный остатки:")
# print(df_estate.iloc[[sresiduals.idxmax(), sresiduals.idxmin()]])

# # Получаем индексы самых больших ошибок
# max_error_idx = sresiduals.idxmax() # Индекс максимального остатка
# min_error_idx = sresiduals.idxmin() # Индекс минимального остатка

# # Используем .loc, чтобы найти строки именно с этими индексами
# outliers = df_estate.loc[[max_error_idx, min_error_idx]]

# print("Строки с аномально большими остатками (выбросы):")
# display(outliers) 

## Решение с использованием Pipeline с применением Target Encoding

#### ПОДГОТОВКА ДАННЫХ ДЛЯ ВСЕХ МОДЕЛЕЙ (SINGLE SOURCE OF TRUTH)

In [ ]:
# ПОДГОТОВКА ДАННЫХ ДЛЯ ВСЕХ МОДЕЛЕЙ (SINGLE SOURCE OF TRUTH)

print("Начинаем подготовку данных для pipeline ML...")

# 1. Удаляем пропуски в цели
df_model = df_estate.dropna(subset=['target_clean']).copy()

# 2. Глобальная очистка выбросов (Одни правила для всех pipelines)
df_model = df_model[
    (df_model['sqft'] < 20000) & (df_model['sqft'] > 10) & 
    (df_model['lotsize'] < 500000) & 
    (df_model['distance_mean'] < 50) 
]

# Сбрасываем индекс
df_model = df_model.reset_index(drop=True)

# 3. Формируем X и y
cols_to_drop = ['target_clean', 'target_clean_log', 'price_sqft', 'street', 'city', 'county'] 
X = df_model.drop(cols_to_drop, axis=1, errors='ignore')
y = df_model['target_clean']

# 4. Split (Фиксируем RANDOM_SEED)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# 5. Приводим типы (для TargetEncoder)
# Используем .loc для явного указания
X_train.loc[:, 'zipcode'] = X_train['zipcode'].astype(str)
X_test.loc[:, 'zipcode'] = X_test['zipcode'].astype(str)

print(f"Данные разделены. Train shape: {X_train.shape}, Test shape: {X_test.shape}")

# НАСТРОЙКА ПРЕПРОЦЕССОРА (Один на всех)

# Группировка признаков
numeric_features = ['sqft', 'lotsize', 'beds', 'baths', 'year_built', 
                    'schools_count', 'rating_mean', 'distance_mean', 'age',
                    'latitude', 'longitude']

categorical_features = ['status', 'propertyType', 'heating', 'cooling', 'parking', 'state']
high_cardinality_features = ['zipcode']

# Фильтр на случай отсутствия колонок
numeric_features = [c for c in numeric_features if c in X.columns]
categorical_features = [c for c in categorical_features if c in X.columns]
high_cardinality_features = [c for c in high_cardinality_features if c in X.columns]

# Создаем трансформеры
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler()) 
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='other')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

target_enc_transformer = Pipeline(steps=[
    ('target_enc', TargetEncoder()) 
])

# Глобальный препроцессор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('target', target_enc_transformer, high_cardinality_features)
    ],
    verbose_feature_names_out=False
)

print("Препроцессор готов к использованию.")

### Pipeline модель с использованием GradientBoostingRegressor, category_encoders

Рассматриваем этот pipeline как базовый (baseline pipeline) среди последующих пайплайнов. 

In [ ]:
# Модель
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(n_estimators=500, max_depth=6, random_state=RANDOM_SEED))
])

# Обучение
print("Обучение модели через Pipeline (GradientBoosting)...")
model_pipeline.fit(X_train, y_train)

# Предсказание
y_test_pred = model_pipeline.predict(X_test)
y_train_pred = model_pipeline.predict(X_train)

# Вывод метрик
print("Метрики модели Pipeline (GradientBoosting):")
metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name="GradientBoostingRegressor (Pipeline)")

# Сохранение в файл
joblib.dump(model_pipeline, './models/GradientBoosting.pkl', compress=3)
print("Модель 'GradientBoosting.pkl' успешно сохранена.")

### Pipeline модель c подбором гиперпараметров для модели XGBoost

Предпринимаем попытку подобрать гиперпараметры для модели XGBoost с помощью Optuna (так как он работает в разы быстрее и эффективнее, чем GridSearch).

In [ ]:
def objective(trial):
    # 1. Параметры для перебора (XGBoost)
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        # Фиксированные параметры
        'tree_method': 'hist',
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'random_state': RANDOM_SEED,
        'verbosity': 0
    }
    
    # 2. Создаем пайплайн с этими параметрами
    trial_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(**param))
    ])
    
    # 3. Делаем подвыборку (20% от данных), чтобы поиск шел быстро
    # Stratify не нужен для регрессии
    X_train_sub, _, y_train_sub, _ = train_test_split(
        X_train, y_train, train_size=0.2, random_state=RANDOM_SEED
    )
    
    # Сбрасываем индексы, чтобы TargetEncoder не ругался на несовпадение
    X_train_sub = X_train_sub.reset_index(drop=True)
    y_train_sub = y_train_sub.reset_index(drop=True)
    
    # 4. Обучение и оценка
    trial_model.fit(X_train_sub, y_train_sub)
    
    # Предсказываем на всем тестовом наборе (X_test), чтобы оценка была честной
    # X_test индексы трогать не нужно, predict с ними справляется
    preds = trial_model.predict(X_test)
    
    return r2_score(y_test, preds)

print("Запуск оптимизации гиперпараметров (Optuna)...")
# Создаем study и запускаем
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15) # 15 попыток будет достаточно для демонстрации

print(f"\nЛучшие параметры: {study.best_params}")
print(f"Лучший R2 на тесте: {study.best_value}")

#### Обучение XGBoost с лучшими параметрами Optuna

In [ ]:
print("Создание финального пайплайна XGBoost с лучшими гиперпараметрами...")

# 1. Берем лучшие параметры из исследования Optuna
final_params = study.best_params.copy()

# 2. Добавляем/перезаписываем технические параметры для стабильности
# Важно: для сохранения универсальности (чтобы работало и на CPU), 
# можно убрать жесткую привязку к cuda при инференсе, если это поддерживается версией
final_params.update({
    'tree_method': 'hist',
    'device': 'cuda' if torch.cuda.is_available() else 'cpu', # Используем GPU если есть
    'random_state': RANDOM_SEED,
    'verbosity': 0
})

# 3. Собираем финальный Пайплайн
# Используем тот же preprocessor, что и везде
tuned_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(**final_params))
])

# 4. Обучаем на X_train
tuned_pipeline.fit(X_train, y_train)

# 5. Проверка метрик
y_test_pred_tuned = tuned_pipeline.predict(X_test)
y_train_pred_tuned = tuned_pipeline.predict(X_train)

#print("\nМетрики модели TUNED XGBOOST PIPELINE")
metrics_func(y_train, y_train_pred_tuned, y_test, y_test_pred_tuned, model_name='OPTUNA TUNED XGBOOST PIPELINE')

# # 6. Сохранение в файл
joblib.dump(tuned_pipeline, './models/tuned_xgboost.pkl', compress=3)
print("Модель 'tuned_xgboost.pkl' успешно сохранена.")

### Pipeline модель с использованием StackingRegressor (rf, Catboost и ridge)

Экспериментальный вариант с использованием StackingRegressor на GPU (GPU-ACCELERATED STACKING - Только для машин с NVIDIA GPU)

In [ ]:
warnings.filterwarnings("ignore", module="daal4py")
warnings.filterwarnings("ignore", message=".*Threading.*")

if USE_GPU:
    print("Запускаем усиленный стекинг на GPU (rf, Catboost и ridge).")
    # 1. Определяем базовые модели
    # RandomForest: работает на CPU, используем все ядра (n_jobs=-1)
    # CatBoost: работает на GPU, что ускорит бустинг в 20-50 раз
    estimators = [
        ('rf', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_SEED, n_jobs=-1)),
        
        ('cb', CatBoostRegressor(
            iterations=1000,       # Аналог n_estimators, для GPU можно ставить больше (стандарт 1000)
            depth=6,               # Аналог max_depth
            learning_rate=0.1,     # Стандартная скорость обучения
            random_seed=RANDOM_SEED,
            task_type="GPU",       # <--- ВКЛЮЧАЕМ ВАШУ ВИДЕОКАРТУ
            devices='0',           # Используем первую карту (RTX 4060)
            verbose=0              # Отключаем вывод логов обучения в консоль
        )),
        
        ('ridge', Ridge())
    ]

    # 2. Создаем Стекинг
    # n_jobs=-1 запустит обучение фолдов параллельно. 
    # CatBoost будет быстро "пролетать" свои задачи на GPU.
    stacking_regressor = StackingRegressor(
        estimators=estimators,
        final_estimator=LinearRegression(),
        n_jobs=1,  
        passthrough=False # Можно поставить True, если хотите передать исходные признаки в финал
    )

    # 3. Пайплайн
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', stacking_regressor)
    ])

    # 4. Обучение
    # Обучение StackingRegressor с GPU-ускорением (CatBoost + RF)
    # Мониторинг: В терминале Ubuntu наберите 'nvidia-smi -l 1', чтобы видеть нагрузку на карту
    model_pipeline.fit(X_train, y_train)

    # 5. Предсказание
    y_test_pred = model_pipeline.predict(X_test)
    y_train_pred = model_pipeline.predict(X_train)

    # 6. Метрики
    metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name='Pipeline STACKING RF CatBoost(GPU) RIDGE')

    # 7. Сохранение
    joblib.dump(model_pipeline, './models/Stacking_rf_catboost_ridge.pkl', compress=3)
    print("Модель 'Stacking_rf_catboost_ridge.pkl' успешно сохранена.")
else:
    print("GPU не найдена или расчет на GPU отключен пользователем. Пропускаем блок Pipeline STACKING RF CatBoost(GPU) RIDGE.")


### Pipeline модели с использованием StackingRegressor с HistGradientBoostingRegressor

Облегченный вариант модели с использованием HistGradientBoostingRegressor

In [ ]:
# Отключаем специфичные предупреждения от Intel daal4py
warnings.filterwarnings("ignore", module="daal4py")
warnings.filterwarnings("ignore", message=".*Threading.*backend is not supported.*")

# HistGradientBoostingRegressor
estimators = [
    ('rf', RandomForestRegressor(n_estimators=50, max_depth=10, random_state=RANDOM_SEED, n_jobs=-1)), # Уменьшил деревья до 50 для скорости
    ('hgb', HistGradientBoostingRegressor(random_state=RANDOM_SEED)), 
    ('ridge', Ridge())
]

stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(),
    cv=3, # Уменьшаем число фолдов с 5 до 3 для скорости
    n_jobs=-1 # Параллелим процесс
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', stacking_regressor)
])

print("Обучение StackingRegressor (RandomForest + HistGradientBoosting + Ridge)...")
model_pipeline.fit(X_train, y_train)

# Предсказание
y_test_pred = model_pipeline.predict(X_test)
y_train_pred = model_pipeline.predict(X_train)

# Вывод метрик
# print("\nМетрики модели StackingRegressor (RandomForest + HistGradientBoosting + Ridge)")
metrics_func(y_train, y_train_pred, y_test, y_test_pred, model_name='Pipeline STACKING RF+histGB+Ridge')

# Сохранение в файл
joblib.dump(model_pipeline, './models/Stacking_rf_histgb_ridge.pkl', compress=3)
print("Модель 'Stacking_rf_histgb_ridge.pkl' успешно сохранена.")


### Pipeline 4 - с использованием StackingRegressor на GPU

Экспериментальный вариант с использованием StackingRegressor на GPU (GPU-ACCELERATED STACKING - Только для машин с NVIDIA GPU)

In [ ]:
# Экспериментальный вариант с использованием StackingRegressor на GPU

if USE_GPU:
    print("Запускаем усиленный стекинг на видеокарте (Double XGBoost).")
    
    # Чтобы стекинг работал, модели должны быть разными.
    # Создадим две вариации XGBoost:
    
    # 1. "Глубокая" модель (ловит сложные зависимости)
    xgb_deep = XGBRegressor(
        n_estimators=3000,
        learning_rate=0.01,      # Очень медленное обучение
        max_depth=10,            # Глубокие деревья
        subsample=0.6,
        colsample_bytree=0.6,
        tree_method='hist',      # GPU режим
        device='cuda',           # Включаем CUDA
        random_state=RANDOM_SEED,
        verbosity=0
    )

    # 2. "Поверхностная" модель (быстрая, ловит основные тренды)
    # Используем её вместо CatBoost, чтобы избежать конфликта версий sklearn
    xgb_shallow = XGBRegressor(
        n_estimators=4000,
        learning_rate=0.05,      # Побыстрее
        max_depth=4,             # Неглубокие деревья
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method='hist',      # GPU режим
        device='cuda',           # Включаем CUDA
        random_state=100,        # Другой seed
        verbosity=0
    )

    # Список моделей
    estimators_gpu = [
        ('xgb_deep', xgb_deep),
        ('xgb_shallow', xgb_shallow)
    ]

    # 3. Создание Стекинга
    stacking_gpu = StackingRegressor(
        estimators=estimators_gpu,
        final_estimator=Ridge(),
        cv=5,       
        n_jobs=1,   # GPU параллелит вычисления, лишние потоки CPU не нужны
        passthrough=False 
    )

    # 4. Сборка пайплайна
    gpu_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor), 
        ('regressor', stacking_gpu)
    ])

    # 5. Обучение
    #print("\n Начинаем обучение GPU Stacking... (это занимает некоторое время)")
    gpu_pipeline.fit(X_train, y_train)

    # 6. Оценка
    y_test_pred_gpu = gpu_pipeline.predict(X_test)
    y_train_pred_gpu = gpu_pipeline.predict(X_train)
    #print("\n Метрики GPU Stacking (xgb_deep + xgb_shallow):")
    metrics_func(y_train, y_train_pred_gpu, y_test, y_test_pred_gpu, model_name='Pipeline STACKING XGB deep + XGB shallow')

    # Сохранение в файл
    joblib.dump(gpu_pipeline, './models/Stacking_xgb_deep_xgb_shallow.pkl', compress=3)
    print("Модель 'Stacking_xgb_deep_xgb_shallow.pkl' успешно сохранена.")
else:
    print("GPU не найдена или расчет на GPU отключен пользователем. Пропускаем блок ускоренного стекинга.")

# =================================================================
# MODEL: Pipeline STACKING XGB deep + XGB shallow
# -----------------------------------------------------------------
# Metric     |                Train |                 Test
# -----------------------------------------------------------------
# MAE        |            70,372.86 |            99,819.26
# MSE        |       16,704,171,132 |       38,687,157,623
# R2         |               0.9277 |               0.8353
# =================================================================


In [ ]:
logging.info("Завершение")

In [ ]:
# Создаем DataFrame из накопленных данных
df_results = pd.DataFrame(EXPERIMENTS_DATA)

# Сортируем по лучшему результату на тесте
if not df_results.empty:
    df_results = df_results.sort_values(by='R2 Score (Test)', ascending=False).reset_index(drop=True)
    
    # # Функция для подсветки
    # def highlight_max(s):
    #     is_max = s == s.max()
    #     return ['background-color: #006400; font-weight: bold' if v else '' for v in is_max]

    print("=== ИТОГОВАЯ ТАБЛИЦА ЭКСПЕРИМЕНТОВ ===")
    # display(df_results.style.apply(highlight_max, subset=['R2 Score (Test)']))  
    display(
        df_results.style.background_gradient(
            cmap='viridis',  # Цветовая карта, которая хорошо выглядит на темном
            subset=['R2 Score (Test)']
        )
    )
    # Опционально: Сохраняем в CSV
    df_results.to_csv('./logs/final_experiment_results.csv', index=False)
else:
    print("Список экспериментов пуст. Проверьте, вызывалась ли функция metrics_func.")

# Заключение и выводы

Проект прошел путь от базовых экспериментов до построения высокоточной модели оценки недвижимости.

### 1. Влияние качества данных
Ключевым драйвером роста точности стала работа с данными, а не перебор алгоритмов:
*   **Legacy Model:** $R^2 \approx 0.63$ (старые геоданные, использовалсь простая обработка с загрузкой данных о ZIP кодах из внешнего источника).
*   **Pipeline Model:** $R^2 \approx 0.80$ (валидированные геоданные через `pgeocode` + `TargetEncoder`).
> **Вывод:** Исправление ошибок в координатах и правильное кодирование почтовых индексов повысили точность на **17%**.

### 2. Результаты экспериментов pipeline:
*   **Baseline (GradientBoosting):** $R^2 = 0.80$, MAE = \$114k. Хороший старт, подтверждающий корректность предобработки.
*   **Optimized (XGBoost + Optuna):** $R^2 = 0.83$, MAE = \$101,7k. Автоматический подбор параметров позволил выжать максимум из одиночной модели, превзойдя как бейзлайн, так и лучшую простую модель.
*   **Advanced (GPU Stacking):** $R^2 = 0.84$, MAE = \$99,8k. Ансамбль моделей дал наилучший результат, но ценой усложнения архитектуры.

### 3. Сравнение архитектур
*   **Одиночная модель (Tuned XGBoost):** $R^2 = 0.83$. Отличный баланс скорости и точности.
*   **Стекинг (Stacking):**
    *   *CPU-версия* (упрощенная) показала падение качества ($0.76$), доказав, что ансамбль слабых моделей проигрывает сильному бустингу.
    *   *GPU-версия* (глубокие деревья) дала лучший результат ($0.835$), пробив психологическую отметку ошибки **MAE < $100k**.

### 4. Итоговая рекомендация модели
*   **Для внедрения (Production):** Рекомендуется **Tuned XGBoost Pipeline** ($R^2=0.83$, MAE=$103,7k$). Он быстрее обучается, легче поддерживается и уступает лидеру всего 1.5%.
*   **Для аналитики (High Precision):** Если вычислительные ресурсы позволяют (есть GPU), можно использовать **Advanced Stacking** ($R^2=0.835$) для экспертной оценки сложных объектов.

### 5. Бизнес-результат
Разработанная система позволяет оценивать недвижимость со средней абсолютной ошибкой (MAE) около **$100,000**, что при медианной цене дома в $320k-$400k является сильным показателем для автоматической оценки (AVM) по всей территории США.

### 6. Итоговая рекомендация для внедрения (Production):

 Рекомендуем внедрить модель **Tuned XGBoost Pipeline** ($R^2=0.83$, файл `tuned_xgboost.pkl`).

**Обоснование:**
*   **Высокая точность:** Уступает сложному стекингу всего 1% ($0.83$ против $0.84$).
*   **Скорость и простота:** Одиночная модель работает быстрее и потребляет меньше ресурсов, чем ансамбль из 10 моделей (в стекинге).
*   **Удобство:** Модель упакована в единый `Pipeline`, который автоматически обрабатывает сырые данные (включая пропуски и кодирование категорий).

*Примечание: Если бизнес-задача требует борьбы за каждую долю процента точности (например, для высокочастотного трейдинга недвижимостью), можно использовать GPU Stacking ($R^2=0.84$, файл `Stacking_xgb_deep_xgb_shallow.pkl`).*


### 7. Рекомендации по использованию модели в бизнесе (Business Value)

Учитывая метрики модели ($MAE \approx \$101.7k$, $R^2 \approx 0.83$) и скорость её работы, предлагаем следующие сценарии внедрения разработанного решения в процессы агентства:

**1. Инструмент для приоритизации сделок (Lead Scoring)**
*   **Сценарий:** Ежедневно в базу попадают тысячи новых объявлений.
*   **Решение:** Модель автоматически оценивает каждый новый объект.
*   **Действие:** Если `Предсказанная цена` > `Цена листинга` (например, на 20%), объект помечается флагом **"Potential Deal"** (Недооценен) и отправляется топ-агентам в первую очередь. Это позволит выкупать выгодные лоты быстрее конкурентов.

**2. Сервис экспресс-оценки для привлечения клиентов (Lead Generation)**
*   **Сценарий:** Владелец дома хочет узнать рыночную стоимость своего жилья.
*   **Решение:** Виджет на сайте агентства "Оцени свой дом за 1 минуту". Пользователь вводит параметры (площадь, зип-код, спальни), модель выдает диапазон цен (например, \$550k $\pm$ 15%).
*   **Ценность:** Сбор контактов (лидов) потенциальных продавцов недвижимости.

**3. "Второе мнение" при назначении цены (Listing Price Advisor)**
*   **Сценарий:** Риелтор договаривается с продавцом о цене выхода на рынок. Продавец часто завышает ожидания.
*   **Решение:** Риелтор использует модель как объективный аргумент: *"Наш ИИ-алгоритм, обученный на 370 000 сделок, показывает, что рыночный коридор для вашего дома — \$600-650k. Если поставим \$800k, мы потеряем время"*.

**4. Контроль аномалий и ошибок ввода**
*   **Сценарий:** Агент ошибся лишним нулем в площади или цене при заполнении карточки.
*   **Решение:** Если предсказание модели отличается от введенной цены более чем на 50%, система блокирует публикацию объявления и просит менеджера проверить данные. Это повысит качество базы данных.

